	Refuerzo de conceptos: 
		- Al volver a trabajar en el proyecto, consolidarás lo que has aprendido y entenderás mejor cómo aplicar diferentes técnicas y herramientas.
	Perfeccionamiento del código: 
		-Puedes refactorizar y optimizar tu código, mejorando la eficiencia y la claridad. Es una buena oportunidad para aplicar mejores prácticas que quizás no pudiste incluir antes.
	Exploración de nuevas ideas: 
		- Con más tiempo, puedes experimentar con diferentes enfoques o añadir características que no habías considerado antes. Por ejemplo, probar distintos algoritmos de machine learning, usar técnicas de visualización más avanzadas, o incluso implementar un pipeline de datos más robusto.
	Portafolio: 
		- Un proyecto más completo y bien elaborado será una excelente adición a tu portafolio, lo que te será útil para futuras oportunidades laborales o académicas.

## 00.01 Importaciones

In [1]:
# Manejo de archivos y tiempo
import os
import json
from time import sleep
from datetime import datetime, timedelta

# NumPy y Pandas
import numpy as np
import pandas as pd
import hashlib

# Visualización
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Escalado
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# GridSearchCV y Validación
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold, StratifiedKFold

# Métricas
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, classification_report

# Modelos clásicos de series temporales
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Modelos de machine learning
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# PyTorch (Alternativa a Keras/TensorFlow)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Guardado y carga de modelos
import joblib

# Manejo de advertencias
import warnings

In [2]:
# Quiero ver todas las columnas sin elípsis.

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## 00.02 Declaración de funciones

In [122]:
def llamar_archivos(ruta_archivo, dato = ''):
    lista = []

    for archivo in os.listdir(ruta_archivo):
        path_archivo = os.path.join(ruta_archivo, archivo)
        df = pd.DataFrame()
        if archivo.startswith(dato) & archivo.endswith('.csv'):
            df = pd.read_csv(path_archivo)
        elif archivo.startswith(dato) & archivo.endswith('.json'):
            df = pd.read_json(path_archivo)
        elif archivo.startswith(dato) & archivo.endswith('.xml'):
            df = pd.read_xml(path_archivo)
        lista.append(df)
            
    return pd.concat(lista, ignore_index = True)

In [4]:
def unificar_dia(df, col_fecha, col_valor):
    if col_valor in df.columns:
        if col_fecha in df.columns:
            try:
                df[col_fecha] = pd.to_datetime(df[col_fecha])
            except:
                return df, f'La columna {col_fecha} no es convertible a datetime. El Dataframe no ha sido modificado.' 
            df = df.groupby(pd.Grouper(key = col_fecha, freq = 'D'))[col_valor].sum().reset_index()
            return df, f'Dataframe creado con éxito'
        else:
            return df, f'La columna {col_fecha} no existe. El Dataframe no ha sido modificado.'
    else:
        return df, f'La columna {col_valor} no existe. El Dataframe no ha sido modificado.'

In [5]:
def libras_a_kilos(peso_libras):
    return round(peso_libras * 0.4536, 3)

###### Función hash.sha256()
- Cuándo se utiliza una función hash como SHA-256, los datos originales se transforman en una cadena de caracteres (el hash) que no puede ser revertida para obtener el dato original.

In [6]:
# 	anonimizar_id: Función que anonimiza los valores de id_autor usando SHA-256.
#   author_id: Se almacenan los IDs de los autores.
#   author_name: Se almacenan los nombres de los autores. Se reemplaza con un alias: “Usuario1”, “Usuario2”...

def anonimizar_valor(valor):
    if pd.isna(valor):  # Si el valor es NaN, devuelve NaN
        return valor
    return hashlib.sha256(str(valor).encode()).hexdigest()


## 01 EDA

### 01.01 EDA Fitbit

#### 01.01.01 Account Changes

- El archivo CSV de Account_Management_Events en los datos exportados de Fitbit generalmente contiene información relacionada con eventos de gestión de la cuenta. Estos eventos suelen registrar actividades como la creación de la cuenta, inicios de sesión, cambios de configuración, sincronización de dispositivos, y otras acciones administrativas realizadas en mi cuenta de Fitbit.

Columnas que se hallan en este archivo CSV y tipo de datos que proporcionan:

- **Timestamp:** 
    - Este campo indica la fecha y hora en que ocurrió el evento. Es útil para rastrear cuándo se realizaron ciertos cambios o actividades en tu cuenta.
- **Event Type:** 
    - Describe el tipo de evento que se registró. Esto podría incluir eventos como “Login”, “Account Created”, “Device Sync”, “Settings Changed”, entre otros.
- **Device:** 
    - Información sobre el dispositivo asociado con el evento, si corresponde. Esto podría incluir el nombre o tipo de dispositivo Fitbit que usaste.
- **User ID:** 
    - Una identificación única asociada a tu cuenta de usuario. Este campo es importante para distinguir eventos en cuentas con múltiples usuarios o dispositivos.
- **IP Address:** 
    - La dirección IP desde la cual se realizó el evento, si está disponible. Esto puede ser útil para identificar desde qué ubicación se accedió a tu cuenta.
- **Location:** 
    - Información geográfica basada en la IP, como país o ciudad, asociada con el evento. Esto te da una idea de dónde ocurrió la actividad.
- **Device OS**: 
    - El sistema operativo del dispositivo utilizado durante el evento (por ejemplo, iOS, Android).
- **Browser:** 
    - Si el evento ocurrió en un navegador web, este campo puede registrar el nombre del navegador (Chrome, Firefox, etc.).
- **App Version:** 
    - Si el evento ocurrió en la aplicación móvil, este campo podría indicar la versión de la aplicación en uso.

In [7]:
path_account_m_e = 'Datos/Fitbit/Account Changes'

account_df = llamar_archivos(path_account_m_e)
account_df.head(4)

,timestamp,event_name,email,location,ip,outcome,reason
0,Wed Oct 11 19:10:57 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
1,Wed Oct 11 18:14:20 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
2,Wed Oct 11 18:13:49 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
3,Wed Oct 11 15:47:00 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",84.125.86.208,SUCCESS,NaN


In [8]:
col_account = {
    'timestamp': 'Fecha y Hora',
    'event_name': 'Evento',
    'email': 'Correo Electrónico',
    'location': 'Ubicación',
    'ip': 'Direccion IP',
    'outcome': 'Resultado',
    'reason': 'Motivo'
}

account_df = account_df.rename(columns = col_account)
account_df.head(4)

,Fecha y Hora,Evento,Correo Electrónico,Ubicación,Direccion IP,Resultado,Motivo
0,Wed Oct 11 19:10:57 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
1,Wed Oct 11 18:14:20 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
2,Wed Oct 11 18:13:49 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",2.155.169.144,SUCCESS,NaN
3,Wed Oct 11 15:47:00 UTC 2023,Update profile data,alexiagm@msn.com,"Vigo, PO, Spain",84.125.86.208,SUCCESS,NaN


#### 01.01.02 Active Zone Minutes (AZM)

- Los “Active Zone Minutes” (minutos en zona activa) son un tipo de dato que Fitbit proporciona para ayudar a entender mejor el nivel de actividad física y su impacto en la salud. Este dato es importante porque refleja el tiempo que se pasa en diferentes zonas de frecuencia cardíaca durante actividades físicas, lo que permite evaluar la intensidad de los ejercicios y su beneficio para la salud cardiovascular.

Columnas que se hallan en este archivo CSV y tipo de datos que proporcionan:

- **Timestamp (Fecha y Hora):**
    -  Este campo indica el momento exacto en que se registraron los minutos en zona activa. Es útil para ver cuándo ocurrieron los períodos de mayor actividad.
- **Zona de Frecuencia Cardíaca:**
	- **Fat Burn Zone (Zona de quema de grasa):**
		- Refleja la cantidad de minutos en los que tu frecuencia cardíaca estuvo en una zona moderada, generalmente entre el 50% y el 70% de tu frecuencia cardíaca máxima. En esta zona, tu cuerpo está utilizando principalmente grasa como fuente de energía.
		- Cardio Zone (Zona de cardio): Representa los minutos en los que tu frecuencia cardíaca estuvo entre el 70% y el 85% de tu frecuencia cardíaca máxima. Este nivel de esfuerzo mejora la capacidad cardiovascular y la eficiencia del corazón.
		- Peak Zone (Zona de pico): Indica los minutos en los que tu frecuencia cardíaca estuvo por encima del 85% de tu frecuencia cardíaca máxima, generalmente durante ejercicios de alta intensidad. Esta zona es efectiva para mejorar la capacidad cardiovascular y quemar calorías rápidamente.
- **Total Active Zone Minutes (Total de minutos en zona activa):** 
	- Este campo refleja la suma de los minutos que pasaste en las distintas zonas de frecuencia cardíaca. Es un indicador global de tu nivel de actividad física intensa y moderada.
- **Duración del Ejercicio:** 
	- Algunos registros también pueden incluir la duración total del ejercicio o actividad física que contribuyó a esos minutos en zona activa.

¿Por qué son importantes estos datos?

Los minutos en zona activa permiten evaluar la intensidad de los ejercicios, algo que solo contar pasos no podría mostrar. Cuanto más tiempo se pase en zonas de mayor frecuencia cardíaca, más beneficios cardiovasculares se obtienen. Fitbit establece objetivos de minutos en zona activa semanales recomendados por organizaciones de salud como la Organización Mundial de la Salud (OMS). Alcanzar estos objetivos está relacionado con la mejora de la salud cardiovascular y la reducción del riesgo de enfermedades crónicas. Además, se pueden usar estos datos para analizar la efectividad de los entrenamientos. Por ejemplo, si se ve que se pasa poco tiempo en zonas de mayor intensidad, se podría ajustar las rutinas para obtener mejores resultados.

In [9]:
path_active_z_m = 'Datos/Fitbit/Active Zone Minutes (AZM)'

active_z_m = llamar_archivos(path_active_z_m)
active_z_m.head(4)

,date_time,heart_zone_id,total_minutes
0,2022-12-01T14:52,FAT_BURN,1
1,2022-12-02T11:59,FAT_BURN,1
2,2022-12-02T12:04,CARDIO,2
3,2022-12-02T14:27,FAT_BURN,1


#### 01.01.03 Activity goals

Tipos de Datos que Puedes Encontrar en “Activity Goals”:
- **Daily Step Goal (Objetivo diario de pasos):** 
	- El número de pasos que te has propuesto caminar cada día. Es uno de los objetivos más comunes que los usuarios de Fitbit establecen para mantenerse activos.
- **Daily Distance Goal (Objetivo diario de distancia):**
	- La distancia que te propones recorrer cada día, medida en kilómetros o millas, dependiendo de tu configuración.
- **Daily Calorie Burn Goal (Objetivo diario de calorías quemadas):**
	- La cantidad de calorías que te has propuesto quemar cada día a través de la actividad física.
- **Daily Active Minutes Goal (Objetivo diario de minutos activos):**
	-  El tiempo que te propones estar activo cada día. Esto se mide en minutos en los que estás realizando actividad física moderada a intensa.
- **Weekly Exercise Goal (Objetivo semanal de ejercicio):** 
	- Una meta que puedes establecer para la cantidad de días por semana en los que planeas realizar ejercicio. Por ejemplo, podrías establecer un objetivo de ejercitarte al menos 5 días a la semana.
- **Daily Floors Climbed Goal (Objetivo diario de pisos subidos):**
	- El número de pisos (tramos de escaleras) que te propones subir cada día.


    

In [10]:
path_activity_goals = 'Datos/Fitbit/Activity Goals'

activity_goals = llamar_archivos(path_activity_goals)
activity_goals.head(4)

,type,frequency,target,result,status,is_primary,start_date,end_date,created_on,edited_on
0,ACTIVE_ZONE_MINUTES_GOAL,DAILY,30.0,NaN,NEW,False,NaN,NaN,2024-04-24T08:00:33.959+00:00,NaN
1,ACTIVE_ZONE_MINUTES_GOAL,WEEKLY,150.0,NaN,NEW,False,NaN,NaN,2024-04-23T09:25:09.334+00:00,NaN
2,CALORIES_BURNED_GOAL,DAILY,2600.0,184.0,INPROGRESS_SO_FAR_MISSED,False,2024-08-09,2024-08-10,2016-11-15T15:15:08.000+00:00,NaN
3,STEPS_GOAL,DAILY,10000.0,131.0,INPROGRESS_SO_FAR_MISSED,False,2024-08-09,2024-08-10,2016-11-15T15:15:08.000+00:00,NaN


#### 01.01.04 Biometrics/Glucosa

El archivo CSV de Biometrics de Fitbit contiene datos relacionados con mediciones biométricas personales que se han registrado en tu cuenta. Estos datos incluyen métricas como peso, porcentaje de grasa corporal, masa muscular, índice de masa corporal (IMC), y otras métricas relacionadas con la composición corporal y la salud general. Estos datos son útiles para monitorear cambios en tu cuerpo a lo largo del tiempo y para evaluar tu progreso hacia objetivos de salud específicos.

Tipos de Datos en el CSV de Biometrics:
- **Date (Fecha):**
	- La fecha en que se registró la medición biométrica. Esto te permite ver cómo han cambiado tus métricas a lo largo del tiempo.
- **Weight (Peso):**
	- Tu peso corporal en la fecha especificada. Este valor puede estar en kilogramos (kg), libras (lbs), o la unidad que hayas configurado en tu cuenta de Fitbit.
- **Body Fat Percentage (Porcentaje de grasa corporal):**
	- El porcentaje de tu peso total que está compuesto por grasa. Es una métrica importante para evaluar la composición corporal.
- **BMI (Body Mass Index, Índice de Masa Corporal):**
	- El índice de masa corporal es una medida que relaciona tu peso con tu altura para categorizar tu estado de salud (bajo peso, peso normal, sobrepeso, etc.).
- **Lean Mass (Masa magra):**
	- La masa magra es el peso de todo tu cuerpo menos la grasa corporal. Incluye músculos, huesos, agua, etc.
- **Muscle Mass (Masa muscular):**
	- La cantidad de masa muscular que tienes, medida en kilogramos o libras. Es una métrica clave para evaluar la fuerza y la salud muscular.
- **Bone Mass (Masa ósea):**
	- La cantidad de masa ósea en tu cuerpo. Este valor generalmente no cambia mucho con el tiempo, pero es importante para evaluar la salud ósea.
- **Hydration (Hidratación):**
	- El porcentaje de agua en tu cuerpo. Mantener un nivel de hidratación adecuado es esencial para la salud general.
- **Resting Heart Rate (Frecuencia cardíaca en reposo):**
	- Tu frecuencia cardíaca promedio mientras estás en reposo. Es un indicador importante de tu condición cardiovascular y tu nivel de forma física.
- **Blood Pressure (Presión arterial):**
	- Tu presión arterial registrada, si se ha medido y sincronizado con Fitbit.
- **Oxygen Saturation (Saturación de oxígeno):**
	- La cantidad de oxígeno en tu sangre, medida en porcentaje. Es importante para evaluar la función respiratoria y la oxigenación general del cuerpo.

In [11]:
path_biometrics = 'Datos/Fitbit/Biometrics'

biometrics = llamar_archivos(path_biometrics)
biometrics.head(4)

,time,value,unit,data_source,measurement_type,medical_codes,tags,no data,min,max,days,enabled
0,2024-02-01T07:43+01:00,101.0,MG_DL,MANUAL,UNSPECIFIED,41604-0:LOINC,fasting,NaN,NaN,NaN,NaN,NaN
1,2024-02-02T08:10+01:00,99.0,MG_DL,MANUAL,UNSPECIFIED,41604-0:LOINC,fasting,NaN,NaN,NaN,NaN,NaN
2,2024-02-03T08:09+01:00,98.0,MG_DL,MANUAL,UNSPECIFIED,41604-0:LOINC,fasting,NaN,NaN,NaN,NaN,NaN
3,2024-02-04T08:37+01:00,98.0,MG_DL,MANUAL,UNSPECIFIED,41604-0:LOINC,fasting,NaN,NaN,NaN,NaN,NaN


#### 01.01.05 Blocked Users
- Este csv no se va a utilizar para no mostrar datos de terceros.

#### 01.01.06 Daily Readiness

El archivo “Daily Readiness” de Fitbit contiene datos relacionados con tu nivel de preparación diaria, una métrica que Fitbit calcula para ayudarte a entender cómo está tu cuerpo preparado para enfrentar el día en términos de actividad física y recuperación. Este indicador se basa en varios factores, como el sueño, la actividad reciente y la frecuencia cardíaca en reposo, para ofrecerte una puntuación que te guía sobre si deberías enfocarte en un día de ejercicio intenso o en uno de recuperación.

- Tipos de Datos en el CSV de Daily Readiness:
    - **Date (Fecha):**
	    La fecha en la que se calculó la puntuación de preparación diaria. Esto te permite ver tu puntuación a lo largo del tiempo.
	- **Readiness Score (Puntuación de preparación):**
	    La puntuación total que indica tu nivel de preparación para el día. Suele ser un valor numérico que va de 0 a 100, donde una puntuación más alta sugiere que estás más preparado para actividades físicas intensas.
	- **Sleep Score Contribution (Contribución de la puntuación de sueño):**
	    Esta métrica muestra cómo ha contribuido la calidad y duración de tu sueño a la puntuación de preparación diaria. El sueño adecuado mejora esta parte de la puntuación.
	- **Activity Score Contribution (Contribución de la puntuación de actividad):**
	    Muestra cómo tu actividad reciente (generalmente la cantidad de ejercicio y esfuerzo físico en los días anteriores) ha afectado tu preparación. Un equilibrio entre actividad y descanso mejora esta puntuación.
	- **Heart Rate Variability Contribution (Contribución de la variabilidad de la frecuencia cardíaca, HRV):**
	    La variabilidad de la frecuencia cardíaca es una medida de las variaciones en el tiempo entre cada latido del corazón. Una HRV alta generalmente indica que tu cuerpo está bien recuperado, lo que aumenta tu puntuación de preparación.
	- **Resting Heart Rate Contribution (Contribución de la frecuencia cardíaca en reposo):**
	    Muestra cómo tu frecuencia cardíaca en reposo influye en tu preparación diaria. Una frecuencia cardíaca más baja generalmente sugiere una mejor condición física y mayor preparación.
	- **Fatigue Contribution (Contribución del nivel de fatiga):**
	    Indica cuánto influye la fatiga en tu preparación. Si tienes un nivel alto de fatiga, esto reducirá tu puntuación de preparación.
	- **Daily Activity Contribution (Contribución de la actividad diaria):**
	    Refleja cómo tu actividad física diaria está afectando tu preparación para los días siguientes.


In [12]:
path_daily_readliness = 'Datos/Fitbit/Daily Readiness'

preparacion_diaria = llamar_archivos(path_daily_readliness)
preparacion_diaria.head(4)

,date,readiness_score_value,readiness_state,activity_subcomponent,sleep_subcomponent,hrv_subcomponent,activity_state,sleep_state,hrv_state,property_type,value,last_update
0,2023-12-31,65.0,HIGH,65.0,92.0,54.0,LOW,HIGH,MEDIUM,NaN,NaN,NaN
1,2023-12-30,11.0,LOW,11.0,98.0,43.0,HIGH,HIGH,MEDIUM,NaN,NaN,NaN
2,2023-12-29,60.0,MEDIUM,60.0,99.0,40.0,MEDIUM,HIGH,MEDIUM,NaN,NaN,NaN
3,2023-12-28,80.0,HIGH,80.0,97.0,48.0,LOW,HIGH,MEDIUM,NaN,NaN,NaN


#### 01.01.07 Discover 

El archivo CSV de Discover de Fitbit generalmente contiene datos relacionados con el uso de la función de descubrimiento de contenido en la plataforma Fitbit. Esta función te sugiere y permite explorar diversos tipos de contenido, como ejercicios, meditación, artículos sobre salud, y otros recursos que pueden ayudarte a mejorar tu bienestar general.

Tipos de Datos en el CSV de Discover:
- **Date (Fecha):**
	La fecha en la que interactuaste con el contenido sugerido en la sección de “Discover”. Esto te permite rastrear cuándo accediste a los diferentes tipos de contenido.
- **Content Title (Título del contenido):**
	El título del contenido que exploraste o al que accediste a través de la sección de “Discover”. Puede ser el nombre de un artículo, una sesión de ejercicio, una meditación guiada, etc.
- **Content Type (Tipo de contenido):**
	Especifica el tipo de contenido que exploraste. Puede incluir categorías como “Ejercicio”, “Meditación”, “Nutrición”, “Sueño”, o “Artículos de salud”. Este campo te ayuda a entender qué tipo de recursos estás utilizando más frecuentemente.
- **Category (Categoría):**
	Una clasificación más amplia del contenido, como “Salud y Bienestar”, “Actividad Física”, “Mindfulness”, etc. Esto agrupa el contenido en temas generales y te ayuda a analizar cuáles son tus áreas de interés principales.
- **Interaction Type (Tipo de interacción):**
	Describe cómo interactuaste con el contenido. Podría incluir acciones como “Visto”, “Completado”, “Guardado para después”, etc. Esto te ayuda a entender no solo qué contenido te interesa, sino también cómo lo consumes.
- **Duration (Duración):**
	La duración del contenido o el tiempo que pasaste interactuando con él. Esto es especialmente relevante para sesiones de ejercicio o meditación.
- **Completion Status (Estado de finalización):**
	Indica si completaste el contenido o no. Esto es útil para ver qué actividades o recursos sigues hasta el final.
- **Link to Content (Enlace al contenido):**
	Si el contenido está disponible en línea, este campo puede incluir un enlace directo para acceder al mismo nuevamente.


In [13]:
path_discover = 'Datos/Fitbit/Discover'

discover_df = llamar_archivos(path_discover)
discover_df.head(4)

,date,id,title,bundle_id,content_type,rating
0,2024-08-01,07ffe695da2fb5c59a84a97d52f7c0e5,The reset,mindfulness,audio,5
1,2024-08-01,10e77f9dff52b35b5a4d948f826780b4,Learn the one-minute meditation,mindfulness,audio,5
2,2024-08-01,a5e1340025a3b6854808cd70ddde268c,Mindful moment meditation,mindfulness,audio,5
3,2024-08-01,a654a90946aa34a3023336fe2a7f758c,Calm and rest,mindfulness,audio,5


#### 01.01.08 Fitbit Care or Programs 
- Sin datos

#### 01.01.09 Friends
- Este csv no se va a utilizar para no mostrar datos de terceros.

#### 01.01.10 Premium

- Mi enfoque principal es analizar los datos de salud y actividad física recopilada mientras era usuario premium. No se necesita incluir user_premiumSubscription y Transaction. Los datos adicionales proporcionados por la suscripción premium (como métricas avanzadas de sueño, readiness score, etc.) aportan datos que se se encontrarán en los CSVs correspondientes a esas métricas, como “Daily Readiness”, “Advanced Sleep”, “Active Zone Minutes”, etc.

#### 01.01.11 Global Export Data

- En la exportación global de datos de Fitbit, los archivos JSON suelen contener una gran variedad de datos estructurados que se dividen en diferentes categorías relacionadas con la actividad física, salud, sueño, y otros aspectos del bienestar que Fitbit monitorea. Los archivos son parte de esta exportación global y pueden incluir datos detallados sobre varios aspectos del uso de Fitbit.

- Resumen general de los datos de estos archivos JSON:
- **Activity Data (Datos de actividad física):**
    - **Pasos:**
    Datos sobre la cantidad de pasos que has dado en diferentes intervalos de tiempo (minutos, horas, días).
	- **Calorías Quemadas:**
    Información sobre las calorías quemadas durante distintas actividades.
	- **Distancia:**
    Registro de la distancia recorrida en diferentes períodos.
	- **Minutos Activos:**
    Datos sobre el tiempo que pasaste haciendo actividad física moderada a intensa.
	- **Elevación:**
    Pisos subidos y metros o pies ascendidos.
	- **Zonas de Frecuencia Cardíaca:**
    Tiempo pasado en diferentes zonas de frecuencia cardíaca durante el ejercicio.
- **Sleep Data (Datos de sueño):**
    - **Duración del sueño:**
    Tiempo total de sueño por noche.
	- **Fases del sueño:**
    Detalles sobre el tiempo pasado en diferentes fases del sueño (ligero, profundo, REM).
	- **Interrupciones:**
    Cantidad y duración de las veces que te despertaste durante la noche.
- **Heart Rate Data (Datos de frecuencia cardíaca):**
    - **Frecuencia cardíaca en reposo:**
    Tu frecuencia cardíaca promedio mientras estás en reposo.
	- **Frecuencia cardíaca durante el ejercicio:**
    Detalles sobre tu frecuencia cardíaca durante actividades físicas específicas.
	- **Variabilidad de la frecuencia cardíaca (HRV):**
     Datos sobre la variabilidad en el tiempo entre latidos, lo que puede indicar tu nivel de recuperación y estrés.
- **Nutrition Data (Datos de nutrición):**
    - **Ingesta de alimentos:**
    Información sobre las calorías y macronutrientes que has consumido, si has registrado tu ingesta de alimentos.
	- **Hidratación:**
    Datos sobre la cantidad de agua que has consumido.
- **Device Information (Información del dispositivo):**
    - **Batería:**
    Nivel de batería de tu dispositivo Fitbit.
	- **Sincronización:**
    Registro de cuándo y cómo se ha sincronizado tu dispositivo con la app de Fitbit.
- **Stress and Mindfulness (Datos de estrés y mindfulness):**
    - **Sesiones de mindfulness:**
    Detalles sobre las sesiones de respiración guiada o meditación que has realizado.
	- **Nivel de estrés:**
    Si tu dispositivo lo permite, puede incluir información sobre tu nivel de estrés medido a través de la variabilidad de la frecuencia cardíaca o sensaciones registradas.
- **Menstrual Health Tracking (Seguimiento de salud menstrual):**
    - **Ciclo menstrual:**
    Información sobre el seguimiento de tu ciclo menstrual, si lo has registrado.
	- **Síntomas:**
    Registro de síntomas relacionados con tu ciclo.
- **Notifications (Notificaciones):**
    - **Notificaciones de salud:**
    Datos sobre las alertas y notificaciones que has recibido relacionadas con tu salud y actividad.
- **User Profile Data (Datos de perfil de usuario):**
    - **Información personal:**
    Datos básicos como edad, sexo, altura, peso (si lo has registrado en la app).


##### altitude

In [14]:
path_global_export_data = 'Datos/Fitbit/Global Export Data'

altitud = llamar_archivos(path_global_export_data, 'altitude')
altitud.head(4)

,dateTime,value
0,2023-01-12 11:11:00,10
1,2023-01-12 11:43:00,20
2,2023-01-12 13:51:00,10
3,2023-01-12 14:07:00,10


##### badge (Insignias)

In [15]:
insignias = llamar_archivos(path_global_export_data, 'badge')
insignias.tail(4)

,encodedId,badgeType,value,timesAchieved,dateTime,name,shortName,earnedMessage,description,category,shareText,unit
47,2297X6,GOAL_BASED_WEIGHT_LOSS,21,12,2024-08-10,-21 kg de pérdida de peso,-21 kg de pérdida de peso,¡Has perdido 21 kg! Enhorabuena por el progreso que has hecho con respecto a tu objetivo de peso.,-21 kilos,Objetivo de peso,He perdido 21 kilos con #Fitbit.,KILOGRAMS
48,2297WZ,GOAL_BASED_WEIGHT_LOSS,18,14,2024-08-10,-18 kg de pérdida de peso,-18 kg de pérdida de peso,¡Has perdido 18 kg! Enhorabuena por el progreso que has hecho con respecto a tu objetivo de peso.,-18 kilos,Objetivo de peso,He perdido 18 kilos con #Fitbit.,KILOGRAMS
49,2297WY,GOAL_BASED_WEIGHT_LOSS,15,16,2024-08-10,-15 kg de pérdida de peso,-15 kg de pérdida de peso,¡Has perdido 15 kg! Enhorabuena por el progreso que has hecho con respecto a tu objetivo de peso.,-15 kilos,Objetivo de peso,He perdido 15 kilos con #Fitbit.,KILOGRAMS
50,22984Y,LIFETIME_WEIGHT_GOAL_SETUP,1,1,2016-11-15,Objetivo de peso configurado,Objetivo de peso configurado,NaN,Objetivo de peso,Objetivo de peso,He establecido un objetivo de peso. #Fitbit,NaN


##### calories (calorias)

In [16]:
calorias = llamar_archivos(path_global_export_data, 'calories')
calorias.sample(4)

,dateTime,value
4067125,2024-01-04 10:42:00,4.06
905996,2018-04-07 03:56:00,1.17
3830316,2021-04-04 23:53:00,1.59
918526,2020-05-04 20:46:00,1.47


##### custom_max_heart_rate (Frecuencia Cardíaca Máxima Personalizada)

- Es el valor específico que se ha configurado como frecuencia cardíaca máxima. Este valor puede ser diferente del valor predeterminado calculado por Fitbit (que usualmente es 220 menos la propia edad).
	
- **Date Set (Fecha de Configuración):**
	La fecha en la que se configuró o actualizó la frecuencia cardíaca máxima personalizada. Esto permite ver cuándo se realizaron cambios en esta configuración.
- **Source (Fuente de Configuración):**
	Indica cómo se estableció este valor. Puede ser configurado manualmente por el usuario a través de la app o en la web de Fitbit, o podría haber sido sugerido por un profesional de la salud y luego ingresado manualmente.
- **User ID (ID de Usuario):**
	El identificador único de la cuenta de Fitbit. Esto asegura que la configuración está asociada con una cuenta específica.


In [17]:
fcm_personalizada = llamar_archivos(path_global_export_data, 'custom_max_heart_rate')
fcm_personalizada.head(4)

,dateTime,value
0,2017-05-08,"{'enabled': False, 'maxHeartRate': 178}"


##### demographic_vo2_max
- Es una medida de la capacidad máxima del cuerpo para consumir oxígeno durante el ejercicio. Es un indicador clave de el nivel de aptitud cardiovascular. Fitbit estima la VO2 Max a partir de los datos de actividad física, como la frecuencia cardíaca y otros factores demográficos como la edad, el peso, y el género.

    - **VO2 Max Estimate (Estimación de VO2 Max):**
	    El valor estimado de VO2 Max. Este valor se mide en mililitros de oxígeno por kilogramo de peso corporal por minuto (ml/kg/min).
	- **Date (Fecha de la Estimación):**
	    La fecha en que se calculó o actualizó la estimación del VO2 Max. Esto permite ver cómo ha cambiado el nivel de aptitud cardiovascular a lo largo del tiempo en el usuario.
	- **Fitness Level (Nivel de Aptitud Física):**
	    Un categorizador basado en el VO2 Max. Esto puede clasificarse como “Muy bajo”, “Bajo”, “Promedio”, “Bueno”, “Excelente”, o “Atleta” dependiendo del rango de VO2 Max y cómo se compara con estándares de salud para el grupo demográfico del usuario.
	- **Demographic Data (Datos Demográficos):**
	    Información adicional sobre los datos demográficos que podrían haberse utilizado para calcular el VO2 Max, como la edad, género, peso, y altura.
	- **Heart Rate Data (Datos de Frecuencia Cardíaca):**
	    Datos relacionados con la frecuencia cardíaca que pueden haber sido utilizados para calcular el VO2 Max, incluyendo la frecuencia cardíaca en reposo y durante el ejercicio.


**Explicación de las Columnas:**

- **dateTime:**
	Esta columna indica la fecha en la que se registraron los datos de VO2 Max. Cada fila corresponde a una estimación realizada en un día específico.
- **value:**
	Esta columna contiene un diccionario con varias claves y valores que representan diferentes estimaciones y errores relacionados con el VO2 Max:
	 - **demographicVO2Max:**
	    Es la estimación del VO2 Max en ml/kg/min. Este valor se basa en los datos demográficos y posiblemente en otros factores de la actividad física y salud.
	•	Ejemplo: 26.731690000000004 significa que, para la fecha dada, el VO2 Max estimado fue de aproximadamente 26.73 ml/kg/min.
	- **demographicVO2MaxError:**
	    Representa el margen de error en la estimación del VO2 Max. En este caso, parece que el margen de error es constante en 3.0, lo que indica que hay una variabilidad esperada de +/- 3 ml/kg/min en la estimación.
	- **filteredDemographicVO2Max:**
	    Es una versión filtrada o ajustada de la estimación de VO2 Max. Puede ser el resultado de aplicar un algoritmo de suavizado o filtrado para eliminar ruidos o variaciones no deseadas en la estimación.
	    Ejemplo: 27.042740000000002 indica el VO2 Max filtrado para la fecha dada.
	- **filteredDemographicVO2MaxError:**
	    Este es el margen de error correspondiente a la estimación filtrada del VO2 Max. En este caso, el margen de error es menor, 0.36310000000000003, lo que sugiere una mayor precisión en esta estimación ajustada.

In [18]:
condicion_fisica = llamar_archivos(path_global_export_data, 'demographic_vo2_max')
condicion_fisica.tail(4)

,dateTime,value
2525,2021-11-09,"{'demographicVO2Max': 26.731690000000004, 'demographicVO2MaxError': 3.0, 'filteredDemographicVO2Max': 27.042740000000002, 'filteredDemographicVO2MaxError': 0.36310000000000003}"
2526,2021-11-10,"{'demographicVO2Max': 26.95981, 'demographicVO2MaxError': 3.0, 'filteredDemographicVO2Max': 27.032700000000002, 'filteredDemographicVO2MaxError': 0.36310000000000003}"
2527,2021-11-11,"{'demographicVO2Max': 26.99172, 'demographicVO2MaxError': 3.0, 'filteredDemographicVO2Max': 27.02774, 'filteredDemographicVO2MaxError': 0.36310000000000003}"
2528,2021-11-12,"{'demographicVO2Max': 26.80789, 'demographicVO2MaxError': 3.0, 'filteredDemographicVO2Max': 27.001130000000003, 'filteredDemographicVO2MaxError': 0.36310000000000003}"


##### distance

- Contiene información sobre la distancia que recorrida a lo largo del tiempo, generalmente medida en kilómetros o millas, dependiendo de las configuraciones en la aplicación de Fitbit. Este archivo suele registrar la distancia diaria que se ha caminado, corrido, o recorrido en cualquier otra actividad física que el dispositivo Fitbit haya monitoreado.

- Puede incluirse datos como:

	- **Date (Fecha):**
	    La fecha específica en la que se registró la distancia recorrida. Esto permite hacer un seguimiento diario de la actividad física.
	- **Distance (Distancia):**
	    La distancia total recorrida en un día determinado. Esta distancia es la suma de todas las actividades físicas en las que el usuario se movió, como caminar, correr, ciclismo, etc.
	- **Activity Type (Tipo de actividad):**
	    Aunque no siempre está presente, algunos registros podrían desglosar la distancia por tipo de actividad, como caminar, correr, andar en bicicleta, etc.
	- **Source (Fuente):**
	    La fuente de la medición, como el modelo específico del dispositivo Fitbit o la aplicación móvil utilizada para registrar la actividad. Este campo puede no estar siempre presente, pero si lo está, indica cómo se capturó el dato.


In [19]:
distancia = llamar_archivos(path_global_export_data, 'distance')
distancia.head(4)

,dateTime,value
0,2023-06-10 22:36:00,0
1,2023-06-10 22:58:00,0
2,2023-06-10 23:05:00,0
3,2023-06-10 23:07:00,550


##### estimated_oxygen_variation (variabilidad extimada oxígeno)

- El archivo estimated_oxygen_variation de Fitbit contiene información relacionada con la variabilidad estimada del oxígeno en sangre durante el sueño. Esta métrica se deriva de los datos recopilados por el sensor de frecuencia cardíaca de el dispositivo Fitbit, que también puede estimar la saturación de oxígeno (SpO2). La variabilidad del oxígeno en la sangre puede ofrecer indicaciones sobre la calidad de la respiración mientras se duerme.

    - **Timestamp (Marca temporal):**
	    - La fecha y hora específica en la que se registró la variabilidad de oxígeno. Esto permite un seguimiento detallado, generalmente a intervalos de minutos, durante el sueño.
            - Indica el momento específico en el que se registró la variación de oxígeno. Los registros suelen estar espaciados a intervalos regulares, lo que permite un análisis detallado del patrón de variabilidad del oxígeno durante el sueño.
	- **Oxygen Variation (Variación de oxígeno):**
	    - Un valor numérico que representa la variación estimada en los niveles de oxígeno en la sangre durante el tiempo registrado. Este valor es una estimación basada en los datos de la frecuencia cardíaca y cómo varía mientras se duerme.
            - Este es el valor estimado de la variabilidad del oxígeno en la sangre. Generalmente, se mide como un porcentaje o una variación relativa, con valores más altos que podrían indicar fluctuaciones en la saturación de oxígeno durante el sueño. Valores consistentes o bajos suelen ser un indicador de una respiración estable.

In [20]:
variacion_oxigeno = llamar_archivos(path_global_export_data, 'estimated_oxygen_variation')
variacion_oxigeno

,timestamp,Infrared to Red Signal Ratio
0,01/02/19 00:36:31,-12
1,01/02/19 00:37:31,-13
2,01/02/19 00:38:31,-2
3,01/02/19 00:39:31,-1
4,01/02/19 00:40:31,-1
...,...,...
759237,01/27/19 23:55:01,27
759238,01/27/19 23:56:01,0
759239,01/27/19 23:57:01,3
759240,01/27/19 23:58:01,0


###### 	Infrared to Red Signal Ratio:
- Esta columna muestra el valor del ratio entre la señal infrarroja y la señal roja capturada por el dispositivo.
- Valores positivos o negativos de este ratio indican la relación entre las señales captadas, que puede estar influenciada por factores como la oxigenación de la sangre y el flujo sanguíneo.
- Por ejemplo, un valor de -12 indica que la señal infrarroja era significativamente menor en relación con la señal roja en ese momento.

##### exercise (ejercicio)

- Contiene información detallada sobre las sesiones de ejercicio registradas por el dispositivo Fitbit. Este archivo es crucial para comprender los hábitos de ejercicio, el tipo de actividades realizadas, y cómo estas afectan la salud y el estado físico general.

- El archivo suele incluir detalles como:
    - **Exercise Name (Nombre del ejercicio):**
	    El nombre del tipo de ejercicio realizado, como “Running” (Correr), “Cycling” (Ciclismo), “Walking” (Caminar), “Yoga”, “Swimming” (Nadar), etc.
	- **Start Time (Hora de inicio):**
	    La fecha y hora exactas en las que se comenzó la sesión de ejercicio. Esto es útil para rastrear cuándo se realiza la mayoría de los entrenamientos.
	- **End Time (Hora de finalización):**
	    La fecha y hora en las que se terminó la sesión de ejercicio, lo que permite calcular la duración total del ejercicio.
	- **Duration (Duración):**
	    La duración de la sesión de ejercicio, generalmente medida en milisegundos. Este dato es clave para entender cuánto tiempo se pasa haciendo ejercicio el usuario en cada sesión.
	- **Calories Burned (Calorías quemadas):**
	    La cantidad de calorías quemadas durante la sesión de ejercicio. Este es un dato importante para entender el impacto de la actividad física en el gasto energético diario.
	- **Distance (Distancia):**
	    Si el ejercicio implica desplazamiento (como correr o caminar), esta columna registra la distancia total recorrida durante la sesión.
	- **Average Heart Rate (Frecuencia cardíaca promedio):**
	    La frecuencia cardíaca promedio durante la sesión de ejercicio. Esto es útil para entender la intensidad del ejercicio y su impacto en el sistema cardiovascular.
	- **Max Heart Rate (Frecuencia cardíaca máxima):**
	    La frecuencia cardíaca máxima alcanzada durante la sesión de ejercicio. Permite ver cuán cerca estuviste del máximo potencial cardiovascular durante el ejercicio.
	- **Steps (Pasos):**
	    El número de pasos dados durante la sesión de ejercicio, si es aplicable (por ejemplo, correr o caminar).
	- **Zones (Zonas de frecuencia cardíaca):**
	    Detalles sobre cuánto tiempo se pasó en diferentes zonas de frecuencia cardíaca, como “Fat Burn” (Quema de grasa), “Cardio”, o “Peak” (Pico). Esto es crucial para entender el tipo de beneficios que se está obteniendo de los entrenamientos.
	- **Elevation Gain (Ganancia de elevación):**
	    La elevación total ganada durante el ejercicio, útil para actividades como ciclismo o senderismo.

		- Explicación de los Datos:

			- **exerciseName:**
				Indica el tipo de ejercicio realizado.
			- **startTime y endTime:**
				Muestran cuándo se comenzó y terminó el ejercicio.
			- **duration:**
				Proporciona la duración de la sesión, que en el ejemplo es de 30 minutos (1800000 milisegundos).
			- **caloriesBurned:**
				Indica cuántas calorías se quemaron durante la sesión, útil para el seguimiento de la energía gastada.
			- **distance:**
				La distancia recorrida durante la sesión (en kilómetros o millas).
			- **averageHeartRate y maxHeartRate:**
				Ofrecen información sobre la intensidad cardiovascular del ejercicio.
			- **steps:**
				Muestra cuántos pasos se dieron durante el ejercicio.
			- **zones:**
				Detalla el tiempo (en segundos) que se pasó en diferentes zonas de frecuencia cardíaca.
			- **elevationGain:**
				Muestra la elevación total ganada, relevante para ejercicios en terrenos con cambios de altitud.


In [21]:
ejercicio = llamar_archivos(path_global_export_data, 'exercise')
ejercicio.tail(4)

,logId,activityName,activityTypeId,activityLevel,averageHeartRate,calories,duration,activeDuration,steps,logType,manualValuesSpecified,heartRateZones,activeZoneMinutes,lastModified,startTime,originalStartTime,originalDuration,elevationGain,hasGps,shouldFetchDetails,hasActiveZoneMinutes,distance,distanceUnit,source,speed,pace,tcxLink,intervalWorkoutData,swimLengths,poolLength,poolLengthUnit
3560,20113132553,Caminar,90013,"[{'minutes': 0, 'name': 'sedentary'}, {'minutes': 0, 'name': 'lightly'}, {'minutes': 0, 'name': 'fairly'}, {'minutes': 19, 'name': 'very'}]",113.0,169,1643000,1081000,2091.0,tracker,"{'calories': False, 'distance': False, 'steps': False}","[{'name': 'Fuera del rangoo', 'min': 30, 'max': 88, 'minutes': 1}, {'name': 'Quema de grasas', 'min': 88, 'max': 123, 'minutes': 17}, {'name': 'Zona cardio', 'min': 123, 'max': 150, 'minutes': 0}, {'name': 'Pico', 'min': 150, 'max': 220, 'minutes': 0}]",NaN,02/25/19 09:15:33,02/25/19 08:42:56,02/25/19 08:42:56,1643000,28.651,True,True,False,1.574210,Kilometer,"{'type': 'tracker', 'name': 'Versa', 'id': '124528151', 'url': 'https://www.fitbit.com/', 'trackerFeatures': ['GPS', 'ELEVATION', 'VO2_MAX', 'PACE', 'STEPS', 'CALORIES', 'DISTANCE', 'HEARTRATE']}",5.242479,686.698005,https://www.fitbit.com/activities/exercise/20113132553?export=tcx,NaN,NaN,NaN,NaN
3561,20112430398,Caminar,90013,"[{'minutes': 0, 'name': 'sedentary'}, {'minutes': 0, 'name': 'lightly'}, {'minutes': 0, 'name': 'fairly'}, {'minutes': 15, 'name': 'very'}]",112.0,132,944000,900000,1762.0,tracker,"{'calories': False, 'distance': False, 'steps': False}","[{'name': 'Fuera del rangoo', 'min': 30, 'max': 88, 'minutes': 0}, {'name': 'Quema de grasas', 'min': 88, 'max': 123, 'minutes': 14}, {'name': 'Zona cardio', 'min': 123, 'max': 150, 'minutes': 1}, {'name': 'Pico', 'min': 150, 'max': 220, 'minutes': 0}]",NaN,02/25/19 11:12:55,02/25/19 10:56:21,02/25/19 10:56:21,944000,5.791,True,True,False,1.324210,Kilometer,"{'type': 'tracker', 'name': 'Versa', 'id': '124528151', 'url': 'https://www.fitbit.com/', 'trackerFeatures': ['GPS', 'ELEVATION', 'VO2_MAX', 'PACE', 'STEPS', 'CALORIES', 'DISTANCE', 'HEARTRATE']}",5.318400,676.895307,https://www.fitbit.com/activities/exercise/20112430398?export=tcx,NaN,NaN,NaN,NaN
3562,20116420208,Ciclismo,90001,"[{'minutes': 0, 'name': 'sedentary'}, {'minutes': 3, 'name': 'lightly'}, {'minutes': 5, 'name': 'fairly'}, {'minutes': 3, 'name': 'very'}]",112.0,74,697000,695000,NaN,tracker,"{'calories': False, 'distance': False, 'steps': False}","[{'name': 'Fuera del rangoo', 'min': 30, 'max': 88, 'minutes': 0}, {'name': 'Quema de grasas', 'min': 88, 'max': 123, 'minutes': 10}, {'name': 'Zona cardio', 'min': 123, 'max': 150, 'minutes': 1}, {'name': 'Pico', 'min': 150, 'max': 220, 'minutes': 0}]",NaN,02/25/19 13:41:29,02/25/19 13:14:14,02/25/19 13:14:14,697000,11.887,True,True,False,1.384421,Kilometer,"{'type': 'tracker', 'name': 'Versa', 'id': '124528151', 'url': 'https://www.fitbit.com/', 'trackerFeatures': ['SPEED', 'GPS', 'ELEVATION', 'CALORIES', 'DISTANCE', 'HEARTRATE']}",7.171096,NaN,https://www.fitbit.com/activities/exercise/20116420208?export=tcx,NaN,NaN,NaN,NaN
3563,20130592937,Ciclismo,90001,"[{'minutes': 1, 'name': 'sedentary'}, {'minutes': 7, 'name': 'lightly'}, {'minutes': 0, 'name': 'fairly'}, {'minutes': 0, 'name': 'very'}]",111.0,46,517000,514000,NaN,tracker,"{'calories': False, 'distance': False, 'steps': False}","[{'name': 'Fuera del rangoo', 'min': 30, 'max': 88, 'minutes': 0}, {'name': 'Quema de grasas', 'min': 88, 'max': 123, 'minutes': 8}, {'name': 'Zona cardio', 'min': 123, 'max': 150, 'minutes': 1}, {'name': 'Pico', 'min': 150, 'max': 220, 'minutes': 0}]",NaN,02/25/19 15:38:41,02/25/19 15:26:08,02/25/19 15:26:08,517000,3.962,True,True,False,1.403835,Kilometer,"{'type': 'tracker', 'name': 'Versa', 'id': '124528151', 'url': 'https://www.fitbit.com/', 'trackerFeatures': ['SPEED', 'GPS', 'ELEVATION', 'CALORIES', 'DISTANCE', 'HEARTRATE']}",9.832342,NaN,https://www.fitbit.com/ac

##### food_logs 

- Contiene información sobre los alimentos registrados en la aplicación de Fitbit. Estos datos son útiles para realizar un seguimiento detallado de la ingesta nutricional diaria, lo que puede ayudar a controlar la dieta y a alcanzar los objetivos de salud y estado físico.

Este archivo suele incluir detalles como:

- **Food Name (Nombre del alimento):**
	El nombre del alimento o bebida que has registrado. Puede ser un alimento específico, una receta, o un alimento genérico.
- **Calories (Calorías):**
	La cantidad de calorías que contiene el alimento registrado. Este es un dato clave para el seguimiento de la ingesta calórica diaria.
- **Macronutrients (Macronutrientes):**
	- Carbohydrates (Carbohidratos): 
		La cantidad de carbohidratos en gramos.
	- Protein (Proteínas): 
		La cantidad de proteínas en gramos.
	- Fat (Grasas): La cantidad de grasas en gramos.
		Estos datos te permiten analizar la proporción de macronutrientes en tu dieta.
	- **Meal Type (Tipo de comida):**
		La categoría de la comida en la que se registró el alimento, como “Breakfast” (Desayuno), “Lunch” (Almuerzo), “Dinner” (Cena), “Snack” (Merienda), etc.
	- **Serving Size (Tamaño de la porción):**
		La cantidad del alimento que has consumido. Esto puede incluir tanto la unidad de medida (por ejemplo, gramos, tazas, porciones) como la cantidad específica.
	- **Date Logged (Fecha de registro):**
		La fecha en la que se registró el alimento. Esto te permite ver qué alimentos consumiste en un día específico.
	- **Nutrients (Nutrientes adicionales):**
		Información adicional sobre otros nutrientes, como fibra, azúcares, vitaminas, y minerales, si estos han sido registrados.
	- **Brand (Marca):**
		Si el alimento es un producto comercial, esta entrada puede incluir la marca del alimento.
	- **Log ID (ID de registro):**
		Un identificador único para cada registro de alimento. Esto es útil para identificar y gestionar entradas específicas.

		- Explicación de los Datos:

			- **foodName:**
				El nombre del alimento que has registrado, como “Apple” o “Chicken Breast”.
			- **calories:**
				Las calorías contenidas en la porción consumida.
			- **carbohydrates, protein, fat:**
				Las cantidades de carbohidratos, proteínas, y grasas en gramos.
			- **mealType:**
				La comida del día en la que se consumió el alimento (Desayuno, Almuerzo, etc.).
			- **servingSize:**
				La cantidad específica del alimento consumido, junto con la unidad de medida.
			_**dateLogged:**
				La fecha en la que se registró el alimento.
			- **brand:**
				La marca del alimento, si corresponde.
			- **logId:**
				Un identificador único para cada entrada, útil para gestionar y rastrear registros específicos.

In [22]:
registro_comida = llamar_archivos(path_global_export_data, 'food_logs')
registro_comida.head(4)

,logId,loggedFood,logDate,favorite,nutritionalValues
0,34400999825,"{'foodId': 2637274, 'locale': 'es_ES', 'name': 'Edulcorante, sacarina', 'brand': '', 'accessLevel': 'PUBLIC', 'units': [229, 226, 180, 147, 389], 'calories': 3, 'amount': 1.0, 'unit': {'id': 229, 'name': 'paquete', 'plural': 'paquetes'}, 'mealTypeId': 2}",2024-07-06,False,NaN
1,34401295035,"{'foodId': 536857383, 'locale': 'es_ES', 'name': 'Café con Leche Descremada', 'brand': '', 'accessLevel': 'PUBLIC', 'units': [204, 16062, 17069, 319, 179, 209, 189, 128, 364, 349, 91, 256, 279, 401, 226, 180, 147, 389], 'calories': 94, 'amount': 1.0, 'unit': {'id': 204, 'name': 'mediano', 'plural': 'medianos'}, 'mealTypeId': 2}",2024-07-06,False,NaN
2,34394962878,"{'foodId': 2637274, 'locale': 'es_ES', 'name': 'Edulcorante, sacarina', 'brand': '', 'accessLevel': 'PUBLIC', 'units': [229, 226, 180, 147, 389], 'calories': 3, 'amount': 1.0, 'unit': {'id': 229, 'name': 'paquete', 'plural': 'paquetes'}, 'mealTypeId': 2}",2024-07-06,False,NaN
3,34394982594,"{'foodId': 536857383, 'locale': 'es_ES', 'name': 'Café con Leche Descremada', 'brand': '', 'accessLevel': 'PUBLIC', 'units': [204, 16062, 17069, 319, 179, 209, 189, 128, 364, 349, 91, 256, 279, 401, 226, 180, 147, 389], 'calories': 94, 'amount': 1.0, 'unit': {'id': 204, 'name': 'mediano', 'plural': 'medianos'}, 'mealTypeId': 2}",2024-07-06,False,NaN


In [23]:
registro_comida.columns

Index(['logId', 'loggedFood', 'logDate', 'favorite', 'nutritionalValues'], dtype='object')

In [24]:
# Cuando hay un JSON con estructuras anidadas (como diccionarios o listas dentro de otros diccionarios), 'pd.json_normalize' descompone esas estructuras complejas y las convierte en un DataFrame. Cada clave en el diccionario anidado se convierte en una columna del DataFrame, y los valores asociados se convierten en las filas correspondientes.
# Aquí tomo la columna 'loggedFood' que es un diccionario que me muestra todos los datos unidos ({'foodId': 2637274, 'locale': 'es_ES', 'name': 'Edulcorante, sacarina', 'brand': '', 'accessLevel': 'PUBLIC', 'units': [229, 226, 180, 147, 389], 'calories': 3, 'amount': 1.0, 'unit': {'id': 229, 'name': 'paquete', 'plural': 'paquetes'}, 'mealTypeId': 2}), me crea nuevas columnas con cada clave y pone su valor en ella para que sea mas fácil luego a la hora de trabajar con cada dato.

registro_comida_normalizado = pd.json_normalize(registro_comida['loggedFood'])
registro_comida_normalizado

,foodId,locale,name,brand,accessLevel,units,calories,amount,mealTypeId,unit.id,unit.name,unit.plural,creatorEncodedId
0,2637274,es_ES,"Edulcorante, sacarina",,PUBLIC,"[229, 226, 180, 147, 389]",3,1.0,2,229,paquete,paquetes,NaN
1,536857383,es_ES,Café con Leche Descremada,,PUBLIC,"[204, 16062, 17069, 319, 179, 209, 189, 128, 364, 349, 91, 256, 279, 401, 226, 180, 147, 389]",94,1.0,2,204,mediano,medianos,NaN
2,2637274,es_ES,"Edulcorante, sacarina",,PUBLIC,"[229, 226, 180, 147, 389]",3,1.0,2,229,paquete,paquetes,NaN
3,536857383,es_ES,Café con Leche Descremada,,PUBLIC,"[204, 16062, 17069, 319, 179, 209, 189, 128, 364, 349, 91, 256, 279, 401, 226, 180, 147, 389]",94,1.0,2,204,mediano,medianos,NaN
4,2637274,es_ES,"Edulcorante, sacarina",,PUBLIC,"[229, 226, 180, 147, 389]",3,1.0,1,229,paquete,paquetes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15687,542264385,es_ES,Plátano Maduro,,PUBLIC,"[204, 18589, 16001, 226, 180, 147, 389]",122,100.0,1,147,gramo,gramos,NaN
15688,536857383,es_ES,Café con Leche Descremada,,PUBLIC,"[204, 16062, 17069, 319, 179, 209, 189, 128, 364, 349, 91, 256, 279, 401, 226, 180, 147, 389]",94,1.0,1,204,mediano,medianos,NaN
15689,727800393,NaN,Ensalada completa Gourmet con nueces y queso de cabra,Florette,PRIVATE,"[226, 180, 147, 389]",203,130.0,5,147,gramo,gramos,535WGF
15690,2642606,es_ES,Bacalao,,PUBLIC,"[304, 226, 180, 147, 389]",150,200.0,5,147,gramo,gramos,NaN


In [25]:
registro_comida_concat = pd.concat([registro_comida.drop(columns = ['loggedFood']), registro_comida_normalizado], axis = 1)
registro_comida_concat.columns

Index(['logId', 'logDate', 'favorite', 'nutritionalValues', 'foodId', 'locale',
       'name', 'brand', 'accessLevel', 'units', 'calories', 'amount',
       'mealTypeId', 'unit.id', 'unit.name', 'unit.plural',
       'creatorEncodedId'],
      dtype='object')

###### Columnas del DataFrame 'registro_comida':

- **foodId:**
	El identificador único para cada alimento en la base de datos de Fitbit. Este ID es útil para identificar alimentos de manera unívoca y podría ser usado para hacer referencias cruzadas si hay otra base de datos de alimentos.
- **locale:**
	El código de localización, que en este caso es es_ES, indicando que el registro está en español (España). Este dato es útil si se trabaja con diferentes idiomas o configuraciones regionales.
- **name:**
	El nombre del alimento registrado, como “Edulcorante, sacarina”, “Café con Leche Descremada”, “Plátano Maduro”, etc. Este es el nombre que ves en la aplicación Fitbit cuando registras la comida.
- **brand:**
	La marca del alimento, si está disponible. Por ejemplo, “Florette” o “Gervais”. Si el alimento no tiene una marca específica registrada, esta columna puede estar vacía.
- **accessLevel:**
	Indica si el alimento es público (PUBLIC) o privado (PRIVATE) en la base de datos de Fitbit. Un alimento PUBLIC puede ser accedido por otros usuarios, mientras que un PRIVATE es específico para el usuario.
- **units:**
	Una lista de unidades en las que el alimento puede ser medido, como gramos, paquetes, unidades, etc. Esta columna aún está en formato de lista, lo que indica que el alimento puede tener múltiples formas de ser medido.
- **calories:**
	La cantidad de calorías en la porción registrada del alimento. Este es un valor importante para el seguimiento de la ingesta calórica diaria.
- **amount:**
	La cantidad específica del alimento consumido. Por ejemplo, 1.0 podría representar 1 paquete, 1 unidad, 100 gramos, etc., dependiendo de la unidad asociada.
- **mealTypeId:**
	Un identificador para el tipo de comida (por ejemplo, desayuno, almuerzo, cena, etc.). Puede usarse este ID para agrupar o filtrar alimentos según el tipo de comida.
- **unit.id:**
	El identificador único de la unidad específica utilizada para medir la cantidad del alimento.
- **unit.name:**
	El nombre de la unidad utilizada, como “paquete”, “mediano”, “gramo”, “unidad”, etc. Este es el nombre singular de la unidad.
- **unit.plural:**
	La forma plural de la unidad, como “paquetes”, “medianos”, “gramos”, “unidades”, etc. Esto puede ser útil si se está generando reportes o textos basados en los datos.
- **creatorEncodedId:**
	Un identificador que podría estar relacionado con el usuario que creó o registró el alimento. En muchos casos, esta columna puede estar vacía (NaN), lo que sugiere que no siempre es relevante.

##### heart_rate 

- Contiene información detallada sobre la frecuencia cardíaca, que es uno de los datos más importantes para monitorear la salud cardiovascular. Fitbit registra la frecuencia cardíaca a intervalos regulares durante todo el día y la noche, permitiéndo analizar cómo varía el pulso en diferentes situaciones, como durante el ejercicio, el descanso o el sueño.

Este archivo generalmente incluye detalles como:

- **DateTime (Fecha y hora):**
	La marca temporal exacta en la que se registró la frecuencia cardíaca. Esto permite un seguimiento continuo y detallado de cómo cambia tu frecuencia cardíaca a lo largo del día.
- **Heart Rate (Frecuencia cardíaca):**
	El valor de la frecuencia cardíaca medido en latidos por minuto (BPM). Este es el dato principal que se analiza para monitorear la salud cardiovascular.
- **Zones (Zonas de frecuencia cardíaca):**
	- **Out of Range:**
		Periodos en los que tu frecuencia cardíaca estaba por debajo de la zona de quema de grasa, generalmente en reposo.
	- **Fat Burn:**
		Zona donde tu frecuencia cardíaca está lo suficientemente elevada como para quemar grasa.
	- **Cardio:**
		Una zona más alta donde tu corazón está trabajando más duro, comúnmente alcanzada durante el ejercicio moderado a intenso.
	- **Peak:**
		La zona más alta, alcanzada durante ejercicios de alta intensidad, donde tu frecuencia cardíaca está cerca de su máximo.
- **Resting Heart Rate (Frecuencia cardíaca en reposo):**
	La frecuencia cardíaca medida en reposo, que es un buen indicador general de tu condición cardiovascular. Generalmente se mide durante los periodos de descanso o sueño.
- **Calories Burned (Calorías quemadas):**
	Estimación de las calorías quemadas durante el periodo de tiempo en el que se registró la frecuencia cardíaca. Esto se basa en la intensidad de la frecuencia cardíaca.
- **Activity Type (Tipo de actividad):**
	Información sobre el tipo de actividad que estabas realizando cuando se registró la frecuencia cardíaca, como correr, caminar, estar en reposo, etc.


	- Explicación de los Datos:

		- **dateTime:**
			Indica el momento exacto en que se registró la frecuencia cardíaca.
		- **heartRate:**
			El valor de la frecuencia cardíaca en ese momento específico, medido en BPM.
		- **zones:**
			Un desglose de las zonas de frecuencia cardíaca y cuánto tiempo se pasó en cada una. Las zonas incluyen outOfRange, fatBurn, cardio, y peak.
		- **restingHeartRate:**
			La frecuencia cardíaca en reposo, medida a lo largo del día o durante el sueño.
		- **caloriesBurned:**
			Una estimación de las calorías quemadas, calculada en función de la frecuencia cardíaca y el tipo de actividad.
		- **activityType:**
			El tipo de actividad que se estaba realizando cuando se registró la frecuencia cardíaca, si está disponible.

In [26]:
ritmo_cardiaco = llamar_archivos(path_global_export_data, 'heart_rate')
ritmo_cardiaco.head(4)

,dateTime,value
0,2022-03-09 23:00:02,"{'bpm': 76, 'confidence': 1}"
1,2022-03-09 23:00:07,"{'bpm': 77, 'confidence': 1}"
2,2022-03-09 23:00:12,"{'bpm': 76, 'confidence': 1}"
3,2022-03-09 23:00:17,"{'bpm': 77, 'confidence': 1}"


##### height (altura)

- Contiene información relacionada con la altura, que es un dato importante en el perfil de salud de cualquier usuario. La altura se utiliza para calcular varias métricas de salud y actividad física, como el índice de masa corporal (IMC), las zonas de frecuencia cardíaca, y la estimación del gasto calórico.

- **DateTime (Fecha y hora):**
	- La fecha y hora en la que se registró o actualizó la altura. Esto permite rastrear cuándo se hizo cambios en este dato del perfil.
- **Height (Altura):**
	- El valor de la altura, que suele estar en centímetros o pies y pulgadas, dependiendo de la configuración de la cuenta. Este es el dato principal que se almacena.

In [27]:
altura = llamar_archivos(path_global_export_data, 'height')
altura.head(4)

,dateTime,value
0,2016-11-15,1670


##### lightly_active_minutes

- Contiene información sobre los minutos pasados en actividades de baja intensidad durante el día. Fitbit clasifica la actividad física en varias categorías de intensidad, y “lightly active” (ligeramente activo) se refiere a aquellas actividades que no son completamente sedentarias pero tampoco son moderadas o vigorosas en términos de esfuerzo físico.

    - Este archivo generalmente incluirá detalles como:
        - **DateTime (Fecha y hora):**
	    La fecha y hora en la que se registraron los minutos de actividad ligera. Aunque el registro puede ser a nivel diario, en algunos casos podría haber registros más granulares (por ejemplo, por hora).
	- **Minutes (Minutos activos):**
	    El número total de minutos pasados en actividades de baja intensidad durante el periodo registrado.

        - **dateTime:**
        Muestra la fecha en la que se registraron los minutos de actividad ligera. En el ejemplo, estos registros son diarios.
	    - **minutes:**
        Indica cuántos minutos estuviste realizando actividades de baja intensidad en el día o periodo especificado.


In [28]:
actividad_ligera = llamar_archivos(path_global_export_data, 'lightly_active_minutes')
actividad_ligera.head(4)

,dateTime,value
0,2022-02-16,306
1,2022-02-17,352
2,2022-02-18,279
3,2022-02-19,342


- **Columnas del DataFrame:**

	- **dateTime:**
		Esta columna muestra la fecha en la que se registraron los minutos de actividad ligera. Cada fila representa un día.
	- **value:**
		Esta columna muestra el número de minutos pasados en actividades de baja intensidad durante un día específico. Por ejemplo, el valor 306 indica que el 16 de febrero de 2022 se estuvo activo ligeramente durante 306 minutos.

##### moderately_active_minutes

- Contiene información sobre los minutos pasados en actividades de intensidad moderada durante el día. Este tipo de actividad se encuentra entre las actividades ligeras y las actividades vigorosas, y es crucial para evaluar el nivel general de actividad física.

    - Este archivo generalmente incluirá detalles como:
        - **DateTime (Fecha y hora):**
	    La fecha y hora en la que se registraron los minutos de actividad moderada. Aunque a menudo se agrupan por día, también podría haber registros más granulares en algunos casos.
	- **Minutes (Minutos activos moderados):**
	    El número total de minutos pasados en actividades de intensidad moderada durante el periodo registrado.


        - **dateTime:**
        Muestra la fecha en la que se registraron los minutos de actividad moderada. En este ejemplo, los registros son diarios.
	    - **minutes:**
        Indica cuántos minutos se estubo realizando actividades de intensidad moderada en el día o periodo especificado. Ejemplos de actividades moderadas incluyen caminatas rápidas, andar en bicicleta a un ritmo moderado, jardinería activa, o ejercicios aeróbicos ligeros.

    - Las actividades moderadamente activas incluyen aquellas que elevan el ritmo cardíaco y respiración, pero aún permiten mantener una conversación. Estas actividades son más intensas que las ligeras, pero no tan extenuantes como las vigorosas.

In [29]:
actividad_moderada = llamar_archivos(path_global_export_data, 'moderately_active_minutes')
actividad_moderada.head(4)

,dateTime,value
0,2021-01-22,16
1,2021-01-23,36
2,2021-01-24,0
3,2021-01-25,21


##### resting_heart_rate

- Contiene información sobre la frecuencia cardíaca en reposo. La frecuencia cardíaca en reposo es un indicador clave de la salud cardiovascular general. Es el número de latidos por minuto que el corazón realiza mientras se está en completo reposo, como cuando se está tranquilo o durmiendo. Una frecuencia cardíaca en reposo más baja generalmente indica una mejor condición física y un corazón que funciona de manera más eficiente.

    - Este archivo generalmente incluye detalles como:
    	- **DateTime (Fecha y hora):**
		La fecha y hora en la que se registró la frecuencia cardíaca en reposo. Aunque generalmente se agrupa por día, algunos dispositivos pueden registrar datos más frecuentemente.
		- **Resting Heart Rate (Frecuencia cardíaca en reposo):**
		El valor de la frecuencia cardíaca en reposo en latidos por minuto (BPM). Este es el dato principal que se registra y es un indicador de la salud cardiovascular.

		- Explicación de los Datos:
			- **dateTime:**
			Indica la fecha en la que se registró la frecuencia cardíaca en reposo. En este ejemplo, se registra diariamente.
			- **value:**
			Este es el valor de la frecuencia cardíaca en reposo, medido en latidos por minuto (BPM). Por ejemplo, un valor de 60 BPM indica que, en promedio, el corazón late 60 veces por minuto mientras se está en reposo.

In [30]:
ritmo_cardiaco_reposo = llamar_archivos(path_global_export_data, 'resting_heart_rate')
ritmo_cardiaco_reposo.head(4)

,dateTime,value
0,2022-11-13,"{'date': '11/13/22', 'value': 60.43638515472412, 'error': 7.150618553161621}"
1,2022-11-14,"{'date': '11/14/22', 'value': 61.77538585662842, 'error': 6.9060611724853525}"
2,2022-11-15,"{'date': '11/15/22', 'value': 60.78312587738037, 'error': 6.8263654708862305}"
3,2022-11-16,"{'date': '11/16/22', 'value': 61.54027080535889, 'error': 6.800091743469238}"


In [31]:
ritmo_cardiaco_reposo.columns

Index(['dateTime', 'value'], dtype='object')

- Desglose de la Estructura del DataFrame:
    - **dateTime:**
	    Esta columna contiene la fecha en la que se registró la frecuencia cardíaca en reposo.
    - **value:**
	•	Esta columna contiene un diccionario con los siguientes elementos:
	•	date: La fecha en un formato diferente ('11/13/22'), que puede o no ser útil dependiendo de cómo quieras manejar las fechas.
	•	value: El valor de la frecuencia cardíaca en reposo en latidos por minuto (BPM).
	•	error: Un valor que representa el error o la incertidumbre en la medición de la frecuencia cardíaca. Este puede ser un valor estándar de desviación o un rango de error.

In [32]:
ritmo_cardiaco_reposo_normalizado = pd.json_normalize(ritmo_cardiaco_reposo['value'])
ritmo_cardiaco_reposo_normalizado.columns

Index(['date', 'value', 'error'], dtype='object')

In [33]:
ritmo_cardiaco_reposo_concat = pd.concat([ritmo_cardiaco_reposo.drop(columns = ['value']), ritmo_cardiaco_reposo_normalizado], axis = 1)
ritmo_cardiaco_reposo_concat.columns

Index(['dateTime', 'date', 'value', 'error'], dtype='object')

##### sedentary_minutes

- Contiene información sobre los minutos pasados en un estado sedentario durante el día. Este tipo de datos es importante para evaluar cuántas horas del día se pasan sin realizar actividad física significativa, lo que es un factor clave en la evaluación de la salud general.

Este archivo generalmente incluirá detalles como:

- **DateTime (Fecha y hora):**
	La fecha en la que se registraron los minutos sedentarios. Aunque a menudo se agrupan por día, es posible que haya registros más detallados.
- **Minutes (Minutos sedentarios):**
	El número total de minutos pasados en un estado sedentario durante el periodo registrado. Esto incluye cualquier periodo en el que no se estuvo realizando una actividad física significativa, como estar sentado o acostado, excepto cuando se está durmiendo.

    - Explicación de los Datos:

	    - **dateTime:**
        Muestra la fecha en la que se registraron los minutos sedentarios. En este ejemplo, los registros son diarios.
	    - **minutes:**
        Indica cuántos minutos se estuvo en un estado sedentario en el día o periodo especificado. Por ejemplo, un valor de 720 minutos (12 horas) indica que se pasó esa cantidad de tiempo sin realizar actividades físicas importantes.

In [34]:
sedentarismo = llamar_archivos(path_global_export_data, 'sedentary_minutes')
sedentarismo.head(4)

,dateTime,value
0,2020-02-27,800
1,2020-02-28,645
2,2020-02-29,683
3,2020-03-01,806


##### sleep (sueño)

- Contiene datos sobre los patrones de sueño. El sueño es una parte crucial de la salud general, y Fitbit monitorea varios aspectos del descanso, lo que permite analizar tanto la cantidad como la calidad del sueño.

Este archivo generalmente incluye detalles como:

- **DateTime (Fecha y hora):**
	La fecha y hora en la que comenzaron y terminaron los registros de sueño. Esto permite identificar los ciclos y patrones de sueño.
- **Duration (Duración):**
	La duración total del sueño en milisegundos. Esto es útil para calcular cuántas horas se durmió en total durante una sesión de sueño.
- **Stages (Etapas del sueño):**
	Desglose de las diferentes etapas del sueño durante una sesión, como:
- **Awake (Despierto):**
		Tiempo pasado despierto durante la noche
	- **Light (Sueño ligero):**
		Tiempo pasado en sueño ligero.
	- **Deep (Sueño profundo):**
		Tiempo pasado en sueño profundo.
	- **REM:**
		Tiempo pasado en la etapa REM del sueño, que es importante para la memoria y el aprendizaje.
- **Efficiency (Eficiencia del sueño):**
	Un porcentaje que representa la eficiencia del sueño, calculado como el tiempo realmente dormido dividido por el tiempo total en la cama.
- **Start Time y End Time:**
	La hora exacta en la que comenzó y terminó la sesión de sueño. Esto ayuda a ver a qué hora el usuario se acuesta y a qué hora se levanta.
- **Minutes Asleep y Minutes Awake:**
	El número total de minutos dormido y despierto durante una sesión de sueño.
- **Time in Bed (Tiempo en cama):**
	El tiempo total en la cama, incluyendo tanto el tiempo dormido como el tiempo despierto.
- **Log ID:**
	Un identificador único para cada registro de sueño, que es útil para identificar y gestionar entradas específicas.

	- Explicación de los Datos:

		- **dateTime:**
			Indica la fecha de la sesión de sueño.
		- **duration:**
			Duración total del sueño en milisegundos.
		- **efficiency:**
			Porcentaje que indica la eficiencia del sueño.
		- **startTime y endTime:**
			Marcan el inicio y el final de la sesión de sueño.
		- **minutesAsleep y minutesAwake:**
			Tiempo dormido y despierto.
		- **timeInBed:**
			Tiempo total en la cama.
		- **stages:**
			Desglose del tiempo en cada etapa del sueño (despierto, sueño ligero, profundo, y REM).
		- **logId:**
			Identificador único de la sesión de sueño.

In [35]:
sueño = llamar_archivos(path_global_export_data, 'sleep')
sueño.head(1)

,logId,dateOfSleep,startTime,endTime,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,type,infoCode,logType,levels,mainSleep
0,29804486402,2020-11-23,2020-11-23T17:20:00.000,2020-11-23T18:33:30.000,4380000,0,44,12,17,73,79,classic,2,auto_detected,"{'summary': {'restless': {'count': 2, 'minutes': 29}, 'awake': {'count': 0, 'minutes': 0}, 'asleep': {'count': 0, 'minutes': 44}}, 'data': [{'dateTime': '2020-11-23T17:20:00.000', 'level': 'asleep', 'seconds': 420}, {'dateTime': '2020-11-23T17:27:00.000', 'level': 'restless', 'seconds': 720}, {'dateTime': '2020-11-23T17:39:00.000', 'level': 'asleep', 'seconds': 2220}, {'dateTime': '2020-11-23T18:16:00.000', 'level': 'restless', 'seconds': 1020}]}",True


In [36]:
sueño_normalizado = pd.json_normalize(sueño['levels'], sep = '_')
sueño_normalizado.columns

Index(['data', 'summary_restless_count', 'summary_restless_minutes',
       'summary_awake_count', 'summary_awake_minutes', 'summary_asleep_count',
       'summary_asleep_minutes', 'shortData', 'summary_deep_count',
       'summary_deep_minutes', 'summary_deep_thirtyDayAvgMinutes',
       'summary_wake_count', 'summary_wake_minutes',
       'summary_wake_thirtyDayAvgMinutes', 'summary_light_count',
       'summary_light_minutes', 'summary_light_thirtyDayAvgMinutes',
       'summary_rem_count', 'summary_rem_minutes',
       'summary_rem_thirtyDayAvgMinutes'],
      dtype='object')

In [37]:
sueño_concat = pd.concat([sueño.drop(columns = ['levels']), sueño_normalizado], axis = 1)
sueño_concat.columns

Index(['logId', 'dateOfSleep', 'startTime', 'endTime', 'duration',
       'minutesToFallAsleep', 'minutesAsleep', 'minutesAwake',
       'minutesAfterWakeup', 'timeInBed', 'efficiency', 'type', 'infoCode',
       'logType', 'mainSleep', 'data', 'summary_restless_count',
       'summary_restless_minutes', 'summary_awake_count',
       'summary_awake_minutes', 'summary_asleep_count',
       'summary_asleep_minutes', 'shortData', 'summary_deep_count',
       'summary_deep_minutes', 'summary_deep_thirtyDayAvgMinutes',
       'summary_wake_count', 'summary_wake_minutes',
       'summary_wake_thirtyDayAvgMinutes', 'summary_light_count',
       'summary_light_minutes', 'summary_light_thirtyDayAvgMinutes',
       'summary_rem_count', 'summary_rem_minutes',
       'summary_rem_thirtyDayAvgMinutes'],
      dtype='object')

In [38]:
sueño_concat.head(1)

,logId,dateOfSleep,startTime,endTime,duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,timeInBed,efficiency,type,infoCode,logType,mainSleep,data,summary_restless_count,summary_restless_minutes,summary_awake_count,summary_awake_minutes,summary_asleep_count,summary_asleep_minutes,shortData,summary_deep_count,summary_deep_minutes,summary_deep_thirtyDayAvgMinutes,summary_wake_count,summary_wake_minutes,summary_wake_thirtyDayAvgMinutes,summary_light_count,summary_light_minutes,summary_light_thirtyDayAvgMinutes,summary_rem_count,summary_rem_minutes,summary_rem_thirtyDayAvgMinutes
0,29804486402,2020-11-23,2020-11-23T17:20:00.000,2020-11-23T18:33:30.000,4380000,0,44,12,17,73,79,classic,2,auto_detected,True,"[{'dateTime': '2020-11-23T17:20:00.000', 'level': 'asleep', 'seconds': 420}, {'dateTime': '2020-11-23T17:27:00.000', 'level': 'restless', 'seconds': 720}, {'dateTime': '2020-11-23T17:39:00.000', 'level': 'asleep', 'seconds': 2220}, {'dateTime': '2020-11-23T18:16:00.000', 'level': 'restless', 'seconds': 1020}]",2.0,29.0,0.0,0.0,0.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### steps (pasos)

- Contiene información sobre la cantidad de pasos dados en diferentes periodos de tiempo. Este es uno de los datos más fundamentales y populares que los dispositivos Fitbit registran, ya que los pasos son una métrica clave para evaluar la actividad física diaria.

- Este archivo generalmente incluye detalles como:

	- **DateTime (Fecha y hora):**
	    La fecha y hora en la que se registraron los pasos. Dependiendo de cómo se agrupen los datos, esto podría ser a nivel diario, por hora, o incluso en intervalos más cortos.
        - 	Indica la fecha (y posiblemente la hora) en la que se registraron los pasos. En muchos casos, estos registros son diarios, pero     también podrían estar en intervalos más cortos si el archivo tiene un seguimiento más granular.
	- **Value (Número de pasos):**
	    El número total de pasos dados durante el periodo especificado por dateTime.
        -   Este es el número de pasos dados en el periodo especificado por dateTime. Por ejemplo, un valor de 10000 indica que se dieron 10,000 pasos en ese día específico.


In [39]:
pasos = llamar_archivos(path_global_export_data, 'steps')
pasos.head(4)

,dateTime,value
0,2023-04-11 22:02:00,0
1,2023-04-11 22:05:00,0
2,2023-04-11 22:07:00,0
3,2023-04-11 22:10:00,0


In [40]:
pasos_sumado, mensaje = unificar_dia(pasos, 'dateTime', 'value')
print(mensaje)

Dataframe creado con éxito


In [41]:
pasos_sumado.head(4)

,dateTime,value
0,2016-11-13,0
1,2016-11-14,9121
2,2016-11-15,9016
3,2016-11-16,3555


##### swim_lengths_data (datos longitud nadada)

- Contiene datos sobre las sesiones de natación, en particular sobre las longitudes (tramos) que se ha nadado. Este archivo es relevante para aquellos que utilizan su Fitbit durante la natación y quieren hacer un seguimiento detallado de sus entrenamientos en el agua.

Este archivo generalmente incluye detalles como:

- **DateTime (Fecha y hora):**
	La fecha y hora en la que se registraron las longitudes nadadas. Dependiendo de cómo se agrupen los datos, esto podría ser a nivel de sesión, por longitud nadada, o en intervalos de tiempo específicos.
- **Length Count (Número de Longitudes):**
	El número de longitudes o tramos nadadas en un periodo específico o durante una sesión.
- **Distance (Distancia):**
	La distancia total que se nadó durante la sesión o periodo especificado. Esto se mide en metros o yardas, dependiendo de la configuración de Fitbit.
- **Duration (Duración):**
	El tiempo total pasado nadando durante la sesión o periodo especificado.
- **Pace (Ritmo):**
	La velocidad promedio a la que se nadó durante la sesión. Esto suele medirse en minutos por 100 metros o una unidad similar.
- **Calories Burned (Calorías Quemadas):**
	Una estimación de las calorías quemadas durante la sesión de natación.

    - Explicación de los Datos:

	- **dateTime:**
        Indica la fecha de la sesión de natación.
	- **lengthCount:**
        Número de longitudes nadadas durante la sesión.
	- **distance:**
        Distancia total nadada, generalmente en metros o yardas.
	- **duration:**
        Tiempo total de la sesión de natación, expresado en segundos.
	- **pace:**
        Ritmo promedio de natación, generalmente en segundos por 100 metros o una unidad similar.
	- **caloriesBurned:**
        Calorías estimadas quemadas durante la sesión de natación.

In [42]:
sesiones_natacion = llamar_archivos(path_global_export_data, 'swim_lengths_data')
sesiones_natacion.head(4)

,dateTime,value
0,2020-01-27 04:40:42,"{'lapDurationSec': 17, 'strokeCount': 6, 'swimStrokeType': 'UNKNOWN', 'swimAlgorithmType': 'EUCLID'}"
1,2020-01-27 05:05:47,"{'lapDurationSec': 19, 'strokeCount': 5, 'swimStrokeType': 'UNKNOWN', 'swimAlgorithmType': 'EUCLID'}"
2,2020-01-27 05:07:48,"{'lapDurationSec': 25, 'strokeCount': 5, 'swimStrokeType': 'UNKNOWN', 'swimAlgorithmType': 'EUCLID'}"
3,2020-01-27 05:08:43,"{'lapDurationSec': 31, 'strokeCount': 5, 'swimStrokeType': 'UNKNOWN', 'swimAlgorithmType': 'EUCLID'}"


In [43]:
sesiones_natacion_normalizado = pd.json_normalize(sesiones_natacion['value'])
natacion_concat = pd.concat([sesiones_natacion.drop(columns = ['value']), sesiones_natacion_normalizado], axis = 1)
natacion_concat.head(4)

,dateTime,lapDurationSec,strokeCount,swimStrokeType,swimAlgorithmType
0,2020-01-27 04:40:42,17,6,UNKNOWN,EUCLID
1,2020-01-27 05:05:47,19,5,UNKNOWN,EUCLID
2,2020-01-27 05:07:48,25,5,UNKNOWN,EUCLID
3,2020-01-27 05:08:43,31,5,UNKNOWN,EUCLID


######
- dateTime: La fecha y hora de cada tramo de natación.
- lapDurationSec: La duración del tramo en segundos.
- strokeCount: El número de brazadas en ese tramo.
- swimStrokeType: El tipo de brazada, aunque en este caso parece que se registra como UNKNOWN.
- swimAlgorithmType: El tipo de algoritmo utilizado para detectar las brazadas, que en este caso es EUCLID.

##### time_in_heart_rate_zones

- Contiene información sobre el tiempo que se pasa en diferentes zonas de frecuencia cardíaca durante las actividades físicas. Las zonas de frecuencia cardíaca son un indicador importante de la intensidad del ejercicio y son utilizadas para medir la eficacia del entrenamiento cardiovascular.

Este archivo generalmente incluye detalles como:

- **DateTime (Fecha y hora):**
	La fecha y hora en la que se registraron los datos de frecuencia cardíaca. Dependiendo de cómo se agrupen los datos, esto podría ser a nivel de sesión, por día, o en intervalos de tiempo específicos.
- **Time in Different Heart Rate Zones (Tiempo en Diferentes Zonas de Frecuencia Cardíaca):**
	- **Fat Burn (Quema de Grasas):**
        El tiempo pasado en la zona de quema de grasas, que suele ser el 50-69% de la frecuencia cardíaca máxima.
    - **Cardio (Cardiovascular):**
        El tiempo pasado en la zona cardio, que suele ser el 70-84% de la frecuencia cardíaca máxima.
	- **Peak (Máximo):**
        El tiempo pasado en la zona de pico, que es el 85% o más de la frecuencia cardíaca máxima.
	- **Out of Zone:**
        El tiempo pasado fuera de cualquier zona específica de frecuencia cardíaca.
- **Total Time in Zones:**
	- El tiempo total pasado en todas las zonas de frecuencia cardíaca combinadas.

        - Explicación de los Datos:

        - **dateTime:**
                Indica la fecha (y posiblemente la hora) en la que se registraron los datos.
        - **fatBurn:**
                El tiempo (en minutos) pasado en la zona de quema de grasas.
        - **cardio:**
                El tiempo (en minutos) pasado en la zona cardiovascular.
        - **peak:**
                El tiempo (en minutos) pasado en la zona de pico de frecuencia cardíaca.
        - **outOfZone:**
                El tiempo (en minutos) pasado fuera de las zonas de frecuencia cardíaca específicas.

In [44]:
frecuencia_cardiaca = llamar_archivos(path_global_export_data, 'time_in_heart_rate_zones')
frecuencia_cardiaca.head(4)

,dateTime,value
0,2018-06-06,"{'valuesInZones': {'IN_DEFAULT_ZONE_3': 0.0, 'IN_DEFAULT_ZONE_2': 0.0, 'IN_DEFAULT_ZONE_1': 150.0, 'BELOW_DEFAULT_ZONE_1': 1178.0}}"
1,2017-10-05,"{'valuesInZones': {'BELOW_DEFAULT_ZONE_1': 1325.0, 'IN_DEFAULT_ZONE_2': 2.0, 'IN_DEFAULT_ZONE_3': 0.0, 'IN_DEFAULT_ZONE_1': 102.0}}"
2,2024-06-04,"{'valuesInZones': {'IN_DEFAULT_ZONE_1': 52.0, 'IN_DEFAULT_ZONE_3': 0.0, 'IN_DEFAULT_ZONE_2': 11.0, 'BELOW_DEFAULT_ZONE_1': 1377.0}}"
3,2021-04-02,"{'valuesInZones': {'IN_DEFAULT_ZONE_3': 0.0, 'IN_DEFAULT_ZONE_2': 0.0, 'BELOW_DEFAULT_ZONE_1': 1390.0, 'IN_DEFAULT_ZONE_1': 50.0}}"


In [45]:
frecuencia_cardiaca_normalizado = pd.json_normalize(frecuencia_cardiaca['value'], sep = '_')
frecuencia_cardiaca_concat = pd.concat([frecuencia_cardiaca.drop(columns = ['value']) , frecuencia_cardiaca_normalizado], axis = 1)
frecuencia_cardiaca_concat.head(4)

,dateTime,valuesInZones_IN_DEFAULT_ZONE_3,valuesInZones_IN_DEFAULT_ZONE_2,valuesInZones_IN_DEFAULT_ZONE_1,valuesInZones_BELOW_DEFAULT_ZONE_1
0,2018-06-06,0.0,0.0,150.0,1178.0
1,2017-10-05,0.0,2.0,102.0,1325.0
2,2024-06-04,0.0,11.0,52.0,1377.0
3,2021-04-02,0.0,0.0,50.0,1390.0


##### very_active_minutes (actividad muy intensa)

- Contiene información sobre los minutos pasados en actividad física muy intensa durante un periodo de tiempo específico. Este tipo de datos es crucial para quienes desean monitorear y mejorar su nivel de actividad física de alta intensidad.

- Este archivo generalmente incluye detalles como:

	- **DateTime (Fecha y hora):**
	    La fecha y, en algunos casos, la hora en la que se registraron los minutos de actividad muy intensa. Dependiendo de cómo se agrupen los datos, esto podría ser un registro diario o incluso más granular.
	- **Value (Minutos de Actividad Muy Intensa):**
	    El número de minutos pasados en una actividad que Fitbit clasifica como “muy activa” durante el periodo especificado. La actividad “muy activa” se refiere a actividades que elevan significativamente la frecuencia cardíaca, como correr, hacer HIIT, o practicar deportes de alta intensidad.

        
- Explicación de los Datos:

	- **dateTime:**
		Indica la fecha (y posiblemente la hora) en la que se registraron los minutos de actividad muy intensa.
	- **value:**
		El número de minutos que se estuvo muy activo durante el periodo especificado.

In [46]:
actividad_intensa = llamar_archivos(path_global_export_data, 'very_active_minutes')
actividad_intensa.head(4)

,dateTime,value
0,2019-01-03,30
1,2019-01-04,68
2,2019-01-05,47
3,2019-01-06,7


- Análisis de la Información:

	- **dateTime:**
        Esta columna  muestra las fechas específicas en las que se realizó actividades de alta intensidad.
	- **value:**
        Aquí se puedes ver cuántos minutos fueron pasados en actividad física muy intensa en cada una de esas fechas.

##### water_logs (agua)

- Contiene datos sobre el consumo de agua registrado a lo largo del tiempo. Este tipo de datos es útil para aquellos que desean llevar un seguimiento de hidratación diaria.

- Este archivo generalmente incluye detalles como:

	- **DateTime (Fecha y hora):**
	    La fecha y, en algunos casos, la hora en la que se registró el consumo de agua.
	- **Amount (Cantidad de Agua Consumida):**
	    La cantidad de agua que se registró haber consumido, normalmente medida en mililitros (ml) o onzas líquidas (fl oz), dependiendo de la configuración.

In [47]:
consumo_agua = llamar_archivos(path_global_export_data, 'water_logs')
consumo_agua.head(4)

,id,date,waterAmount,measurementUnit
0,4008192534,2017-03-09,750,MILLILITER
1,4009092260,2017-03-09,250,MILLILITER
2,4012076875,2017-03-10,500,MILLILITER
3,4012081138,2017-03-10,200,MILLILITER


In [48]:
agua, mensaje =  unificar_dia(consumo_agua, 'date', 'waterAmount')
mensaje

'Dataframe creado con éxito'

In [49]:
agua.head(4)

,date,waterAmount
0,2016-11-15,2250
1,2016-11-16,1250
2,2016-11-17,2000
3,2016-11-18,0


##### weight (peso)

- Contiene información sobre el peso registrado a lo largo del tiempo. Este archivo es especialmente útil para aquellos que están monitoreando su peso como parte de un programa de salud o fitness.

- 	Este archivo generalmente incluye detalles como:

	- **DateTime (Fecha y hora):**
	    La fecha y, en algunos casos, la hora en la que se registró el peso.
	- **Weight (Peso):**
	    El peso registrado, generalmente medido en kilogramos (kg) o libras (lbs), dependiendo de la configuración de la cuenta de Fitbit.
	- **BMI (Índice de Masa Corporal):**
	    El índice de masa corporal asociado con el peso registrado. Este es un cálculo que utiliza el peso y altura del usuario para estimar si se tiene un peso saludable.
	- **Fat (Porcentaje de Grasa Corporal):**
	    El porcentaje de grasa corporal registrado, si se ha medido y registrado junto con el peso.

In [50]:
peso = llamar_archivos(path_global_export_data, 'weight')
peso.head(4)

,logId,weight,bmi,date,time,source
0,1723287310000,226.4,36.82,2024-08-10,10:55:10,API
1,1527033599000,220.9,35.93,2018-05-22,23:59:59,API
2,1527206399000,219.3,35.68,2018-05-24,23:59:59,API
3,1645142399000,285.0,46.36,2022-02-17,23:59:59,API


###### 
- Análisis de la Información:

	- **Peso y BMI:**
        Estas dos columnas dan una idea clara del peso en diferentes momentos y cómo este se relaciona con el índice de masa corporal. Esto puede ser útil para monitorear la evolución del peso y cómo se correlaciona con la salud general.
	- **Fecha y Hora:**
        Saber exactamente cuándo se tomó cada medida permite correlacionar estos datos con otros factores, como los hábitos alimenticios, ejercicio o patrones de sueño.
	- **Fuente del Registro:**
        La columna source indica cómo se registró el peso, lo que puede ser útil si se necesitas filtrar o analizar los datos en función de cómo se obtuvieron (por ejemplo, manualmente vs. automáticamente).

In [51]:
peso['kilos'] = peso['weight'].apply(libras_a_kilos)
peso.head(4)

,logId,weight,bmi,date,time,source,kilos
0,1723287310000,226.4,36.82,2024-08-10,10:55:10,API,102.695
1,1527033599000,220.9,35.93,2018-05-22,23:59:59,API,100.200
2,1527206399000,219.3,35.68,2018-05-24,23:59:59,API,99.474
3,1645142399000,285.0,46.36,2022-02-17,23:59:59,API,129.276


#### 01.01.12 Guided Programs

- Sin datos

#### 01.01.13 Heart Rate

- **Notifications Profile:**
    Contiene configuraciones y ajustes relacionados con tu perfil de notificaciones de frecuencia cardíaca.
- **Notifications Alert:**
    Registra las alertas de frecuencia cardíaca que se han activado, incluyendo el momento y la razón.

##### Notifications Profile

- Este archivo probablemente contiene información sobre el perfil de notificaciones de frecuencia cardíaca, configuraciones y ajustes relacionados con cómo se notifican eventos de frecuencia cardíaca específicos.

- Este archivo generalmente incluye detalles como:	
	- **User ID o Perfil ID:**
		Identificador del usuario o perfil relacionado con las notificaciones.
	- **Thresholds (Umbrales):**
		Los umbrales de frecuencia cardíaca que se han establecido para que se te notifique (por ejemplo, cuando tu frecuencia cardíaca supera o cae por debajo de ciertos límites).
	- **Notification Settings (Configuraciones de Notificaciones):**
		Configuraciones sobre cómo y cuándo recibir las notificaciones (por ejemplo, alertas de vibración, notificaciones push en la app, etc.).
	- **Zones:**
		Las zonas de frecuencia cardíaca personalizadas que has configurado, como la zona de quema de grasa, cardio, y pico.

In [52]:
path_hr_notifications = 'Datos/Fitbit/Heart Rate'

notificaciones_perfil = llamar_archivos(path_hr_notifications, 'Heart Rate Notifications Profile')
notificaciones_perfil.head(4)

,threshold_high_custom,threshold_low_custom,use_custom_threshold_high,use_custom_threshold_low,alert_high_on,alert_low_on
0,130,40,True,True,True,True


##### Notifications Alert

- Este archivo generalmente incluye detalles como:
    - **Timestamp (Marca de Tiempo):**
        El momento exacto en que se activó una alerta de frecuencia cardíaca.
	- **Alert Type (Tipo de Alerta):**
        Descripción de la alerta que se activó (por ejemplo, “frecuencia cardíaca alta”, “frecuencia cardíaca baja”).
	- **Heart Rate (Frecuencia Cardíaca):**
        La frecuencia cardíaca registrada en el momento en que se activó la alerta.
	- **Zone (Zona de Frecuencia Cardíaca):**
        La zona en la que se encontraba tu frecuencia cardíaca al momento de la alerta.
	- **Device Info (Información del Dispositivo):**
        Datos sobre el dispositivo que registró la alerta, como el tipo de dispositivo y su estado de batería o conectividad.

In [53]:
path_hr_notifications = 'Datos/Fitbit/Heart Rate'

notificaciones_alertas = llamar_archivos(path_hr_notifications, 'Heart Rate Notifications Alerts')
notificaciones_alertas.head(4)

,id,start_timestamp,end_timestamp,type,threshold,value


#### 01.01.14 Heart Rate Variability

In [54]:
path_h_r_v = 'Datos/Fitbit/Heart Rate Variability'

##### Daily Heart Rate Variability Summary

- Contiene información relacionada con la variabilidad de la frecuencia cardíaca (HRV). La HRV es una métrica importante que refleja la variabilidad en el tiempo entre los latidos del corazón, lo cual está relacionado con el funcionamiento del sistema nervioso autónomo.

- Este archivo suele incluir los siguientes tipos de datos:

	- **Date (Fecha):**
	    La fecha específica en la que se calculó la variabilidad de la frecuencia cardíaca.
	- **SDNN (Standard Deviation of NN intervals):**
	    Una métrica que mide la variabilidad de la frecuencia cardíaca. Específicamente, es la desviación estándar de los intervalos NN (el tiempo entre latidos sucesivos del corazón). Un SDNN más alto generalmente indica una mejor capacidad de respuesta del sistema nervioso autónomo.
	- **RMSSD (Root Mean Square of Successive Differences):**
	    Otra métrica de HRV que calcula la media cuadrática de las diferencias entre latidos sucesivos. Es particularmente útil para evaluar la actividad del sistema nervioso parasimpático.
	- **LF/HF Ratio:**
	    La relación entre las bandas de baja frecuencia (LF) y alta frecuencia (HF) de la HRV. Este valor puede ofrecer una indicación del equilibrio entre el sistema nervioso simpático y parasimpático.
	- **Resting Heart Rate (Frecuencia Cardíaca en Reposo):**
	    La frecuencia cardíaca promedio medida durante los periodos de descanso.

In [55]:
variabilidad_cardiaca_diaria = llamar_archivos(path_h_r_v, 'Daily Heart Rate Variability Summary')
variabilidad_cardiaca_diaria.head(4)

,timestamp,rmssd,nremhr,entropy
0,2021-04-16T00:00:00,28.435,72.956,2.239
1,2024-03-09T00:00:00,52.863,46.595,2.670
2,2023-04-08T00:00:00,61.370,55.852,2.852
3,2021-05-16T00:00:00,40.832,66.195,2.380


##### Daily Respiratory Rate Summary

- Contiene información relacionada con la frecuencia respiratoria diaria. Esta métrica se refiere al número de respiraciones por minuto que se realizan, y es una medida importante para evaluar la salud respiratoria y cardiovascular.

- Este archivo generalmente incluye los siguientes tipos de datos:

	- **Date (Fecha):**
	    La fecha específica en la que se registró la frecuencia respiratoria.
	- **Breaths per Minute (Respiraciones por Minuto):**
	    El número de respiraciones realizadas por minuto, generalmente durante un periodo de descanso o sueño.
	- **Resting Respiratory Rate (Frecuencia Respiratoria en Reposo):**
	    La frecuencia respiratoria registrada cuando se está en reposo, lo cual es un indicador importante de la salud general.
	- **Respiratory Rate during Sleep (Frecuencia Respiratoria Durante el Sueño):**
	    Puede incluir la frecuencia respiratoria promedio durante diferentes etapas del sueño.

In [56]:
datos_respiraciones = llamar_archivos(path_h_r_v, 'Daily Respiratory Rate Summary')
datos_respiraciones.head(4)

,timestamp,daily_respiratory_rate
0,2023-05-23T00:00:00,19.6
1,2023-02-16T00:00:00,17.8
2,2023-02-02T00:00:00,19.2
3,2021-04-20T00:00:00,18.4


- **timestamp:**
    La fecha y hora en que se registró la frecuencia respiratoria diaria.
- **daily_respiratory_rate:**
    La tasa respiratoria diaria, medida en respiraciones por minuto.

In [57]:
suma_respiraciones, mensaje = unificar_dia(datos_respiraciones, 'timestamp', 'daily_respiratory_rate')
mensaje

'Dataframe creado con éxito'

In [58]:
suma_respiraciones.head(4)

,timestamp,daily_respiratory_rate
0,2020-11-19,18.0
1,2020-11-20,20.6
2,2020-11-21,16.6
3,2020-11-22,17.4


##### Heart Rate Variability Details

- Contiene detalles más granulares sobre la variabilidad de la frecuencia cardíaca (HRV). Mientras que otros archivos pueden ofrecer resúmenes diarios de la HRV, este archivo probablemente proporciona datos más detallados que pueden incluir información minuto a minuto, hora a hora, o incluso latido a latido, dependiendo de cómo se registren los datos.

- Este archivo podría incluir las siguientes columnas:

	- **Timestamp (Marca de Tiempo):**
	    La fecha y hora específica en la que se realizó cada medición de la variabilidad de la frecuencia cardíaca.
	- **SDNN (Standard Deviation of NN intervals):**
	    La desviación estándar de los intervalos NN (el tiempo entre latidos sucesivos del corazón). Este es un indicador de la variabilidad de la frecuencia cardíaca en un periodo de tiempo corto, reflejando la actividad del sistema nervioso autónomo.
	- **RMSSD (Root Mean Square of Successive Differences):**
	    La media cuadrática de las diferencias entre latidos sucesivos. Es una métrica que evalúa la actividad del sistema nervioso parasimpático y es útil para monitorear la recuperación y el estrés.
	- **NN50/PNN50:**
	    NN50 es el número de pares sucesivos de intervalos NN que difieren por más de 50 ms. PNN50 es el porcentaje de intervalos NN que cumplen esta condición. Estos valores son indicadores de la variabilidad de la frecuencia cardíaca.
	- **HF/LF Ratio (Relación de Alta Frecuencia a Baja Frecuencia):**
	    La relación entre las bandas de alta frecuencia (HF) y baja frecuencia (LF) de la variabilidad de la frecuencia cardíaca. Es una métrica utilizada para evaluar el equilibrio entre la actividad del sistema nervioso simpático y parasimpático.

In [59]:
variacion_cardias = llamar_archivos(path_h_r_v, 'Heart Rate Variability Details')
variacion_cardias.head(4)

,timestamp,rmssd,coverage,low_frequency,high_frequency
0,2023-05-28T00:45:00,51.642,0.961,483.913,386.622
1,2023-05-28T00:50:00,44.668,0.937,242.109,319.299
2,2023-05-28T00:55:00,42.858,0.997,100.407,275.355
3,2023-05-28T01:00:00,41.362,1.001,73.206,287.213


##### Heart Rate Variability Histogram

- Proporciona una visión detallada de cómo se distribuyen los valores de HRV en diferentes intervalos (bins). Este tipo de análisis es crucial para comprender mejor la variabilidad de la frecuencia cardíaca y cómo está relacionada con la salud general.

- **timestamp:**
    Indica la fecha y hora en la que se registraron los datos de variabilidad de la frecuencia cardíaca. Esto dice cuándo se tomó cada conjunto de mediciones.
    - Proporciona la marca temporal para cada conjunto de datos de HRV, permitiéndo correlacionar estos datos con otros eventos o períodos de tiempo específicos.
- **bucket_values:**
    Esta columna contiene una lista de valores que representan los recuentos normalizados o las proporciones de los datos de HRV en diferentes rangos (bins). Estos valores son los que formarían el histograma para ese período específico de tiempo.
    - Cada lista en esta columna representa la distribución de la HRV en diferentes intervalos (bins). Estos valores podrían corresponder a frecuencias de HRV divididas en varios rangos. Por ejemplo, si la lista contiene 29 valores, esos 29 valores podrían representar la proporción de tiempo que el HRV pasó en 29 diferentes rangos de frecuencia durante el período especificado.

In [60]:
histograma_cardias = llamar_archivos(path_h_r_v, 'Heart Rate Variability Histogram')
histograma_cardias.head(4)

,timestamp,bucket_values
0,2022-09-01T07:12:30,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.006, 0.017, 0.048, 0.136, 0.225, 0.234, 0.15, 0.097, 0.061, 0.023, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,2022-09-02T07:06:30,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.001, 0.001, 0.002, 0.009, 0.033, 0.1, 0.199, 0.251, 0.261, 0.124, 0.018, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,2022-09-03T07:32:30,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.002, 0.004, 0.026, 0.12, 0.212, 0.21, 0.161, 0.111, 0.071, 0.045, 0.025, 0.009, 0.002, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,2022-09-04T04:51:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001, 0.004, 0.04, 0.131, 0.231, 0.22, 0.186, 0.138, 0.041, 0.007, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


##### Respiratory Rate Summary

- Contiene un resumen de los datos relacionados con la frecuencia respiratoria. Este tipo de datos es crucial para evaluar la salud respiratoria, ya que la frecuencia respiratoria es un indicador importante del bienestar general y puede estar relacionado con el estrés, la actividad física y la calidad del sueño.

- Este archivo podría incluir las siguientes columnas:

	- **timestamp:**
        La fecha y hora específica en la que se registró la frecuencia respiratoria.
	- **respiratory_rate:**
        La frecuencia respiratoria medida en respiraciones por minuto durante un periodo determinado.
	- **min_resp_rate:**
        La frecuencia respiratoria mínima registrada durante el periodo.
	- **max_resp_rate:**
        La frecuencia respiratoria máxima registrada durante el periodo.
	- **mean_resp_rate:**
        La frecuencia respiratoria promedio calculada durante el periodo.

In [61]:
respiraciones_sueno = llamar_archivos(path_h_r_v, 'Respiratory Rate Summary')
respiraciones_sueno.head(4)

,timestamp,full_sleep_breathing_rate,full_sleep_standard_deviation,full_sleep_signal_to_noise,deep_sleep_breathing_rate,deep_sleep_standard_deviation,deep_sleep_signal_to_noise,light_sleep_breathing_rate,light_sleep_standard_deviation,light_sleep_signal_to_noise,rem_sleep_breathing_rate,rem_sleep_standard_deviation,rem_sleep_signal_to_noise
0,2021-02-02T04:57:00,18.4,1.5,6.688,18.0,0.9,6.402,NaN,NaN,NaN,14.6,1.9,3.729
1,2021-02-03T05:14:30,17.8,2.6,13.316,17.8,2.6,13.316,NaN,NaN,NaN,15.2,1.8,6.253
2,2021-02-06T03:51:00,18.4,1.9,6.327,16.8,1.7,8.213,NaN,NaN,NaN,17.4,0.6,2.750
3,2021-02-07T04:15:30,19.2,1.1,5.110,18.6,2.6,11.486,NaN,NaN,NaN,16.2,1.5,4.223


###### 
- Columnas:
    - **timestamp:**
        La fecha y hora en la que se registraron los datos de la frecuencia respiratoria.
	- **full_sleep_breathing_rate:**
        La frecuencia respiratoria promedio durante todo el periodo de sueño.
	- **full_sleep_standard_deviation:**
        La desviación estándar de la frecuencia respiratoria durante todo el sueño, que indica la variabilidad en la respiración.
	- **full_sleep_signal_to_noise:**
        La relación señal/ruido (SNR) durante todo el sueño, que mide la calidad de la señal registrada.
	- **deep_sleep_breathing_rate:**
        La frecuencia respiratoria promedio durante la fase de sueño profundo.
	- **deep_sleep_standard_deviation:**
        La desviación estándar de la frecuencia respiratoria durante el sueño profundo.
	- **deep_sleep_signal_to_noise:**
        La relación señal/ruido durante el sueño profundo.
	- **light_sleep_breathing_rate:**
        La frecuencia respiratoria promedio durante la fase de sueño ligero.
	- **light_sleep_standard_deviation:++
        La desviación estándar de la frecuencia respiratoria durante el sueño ligero.
	- **light_sleep_signal_to_noise:
        La relación señal/ruido durante el sueño ligero.
	- **rem_sleep_breathing_rate:**
        La frecuencia respiratoria promedio durante la fase de sueño REM.
	- **rem_sleep_standard_deviation:**
        La desviación estándar de la frecuencia respiratoria durante el sueño REM.
	- **rem_sleep_signal_to_noise:**
        La relación señal/ruido durante el sueño REM.

#### 01.01.15 HFD_GoogleData

##### GoalSettingsHistory

- Contiene un historial de los objetivos que se han establecido en relación con la salud y actividad física a lo largo del tiempo. Estos objetivos pueden incluir metas diarias o semanales relacionadas con pasos, calorías quemadas, distancia recorrida, frecuencia cardíaca, entre otros.

- Podría incluir las siguientes columnas:
    - **timestamp:**
        La fecha y hora en que se estableció o modificó un objetivo.
	- **goal_type:**
        El tipo de objetivo que se ha establecido, como pasos, calorías, minutos de actividad, etc.
	- **goal_value:**
        El valor del objetivo establecido, como un número específico de pasos, una cantidad de calorías a quemar, etc.
	- **status:**
        El estado del objetivo, que podría indicar si el objetivo está activo, completado, modificado, o eliminado.
	- **source:**
        La fuente o aplicación desde donde se estableció el objetivo (por ejemplo, Google Fit, un dispositivo específico, etc.).

In [62]:
path_HFD_GoogleData = 'Datos/Fitbit/HFD_GoogleData'

metas = llamar_archivos(path_HFD_GoogleData, 'GoalSettingsHistory')
metas.head(4)

,name,objectives,schedule,status,update_time
0,weekly-exercise-days,"MEASURE: EXERCISE_DAYS, TARGET: AT_LEAST(3)",WEEKLY,ENABLED,1970-01-01 00:00:00+0000


######
- **name:**
    El nombre del objetivo o meta establecido. *weekly-exercise-days* el objetivo se refiere al número de días de ejercicio que se planea realizar semanalmente.
- **objectives:**
    Detalles específicos del objetivo: 
    - **MEASURE:**
        *EXERCISE_DAYS, TARGET: AT_LEAST(3)* indica que el objetivo es realizar al menos 3 días de ejercicio por semana.
- **schedule:**
    La frecuencia con la que se evalúa o se espera que se cumpla el objetivo. En este caso, WEEKLY significa que el objetivo se mide semanalmente.
- **status:**
    El estado actual del objetivo. ENABLED indica que este objetivo está actualmente activo y en uso.
- **update_time:**
    La fecha y hora en que se actualizó o se estableció por última vez este objetivo. Sin embargo, en tu caso, la fecha parece ser 1970-01-01 00:00:00+0000, que es un valor comúnmente conocido como el Unix Epoch. Este valor puede sugerir que no se ha registrado una actualización real o que la información de tiempo no se ha configurado correctamente.

##### UserExercises

- Contiene datos relacionados con los ejercicios realizados y registrados a través de la aplicación o dispositivo que está generando estos datos.

- Podría incluir las siguientes columnas:

	- **timestamp:**
		La fecha y hora en que se realizó el ejercicio o se registró en la aplicación.
	- **exercise_type:**
		El tipo de ejercicio realizado, como correr, nadar, andar en bicicleta, yoga, etc.
	- **duration:**
		La duración del ejercicio, normalmente en minutos o segundos.
	- **calories_burned:**
		La cantidad de calorías quemadas durante el ejercicio.
	- **distance:**
		La distancia recorrida durante el ejercicio, especialmente relevante para actividades como correr o andar en bicicleta.
	- **heart_rate:**
		Los datos de frecuencia cardíaca promedio o máxima durante el ejercicio, si están disponibles.
	- **intensity:**
		Una medida de la intensidad del ejercicio, que puede estar basada en zonas de frecuencia cardíaca o en otro tipo de clasificación.
	- **steps:**
		El número de pasos dados durante el ejercicio, si corresponde.
	- **location:**
		Información sobre el lugar donde se realizó el ejercicio, si se trata de una actividad al aire libre.


In [63]:
path_HFD_GoogleData = 'Datos/Fitbit/HFD_GoogleData'

ejercicios_usuario = llamar_archivos(path_HFD_GoogleData, 'UserExercises')
ejercicios_usuario.head(4)

,exercise_id,exercise_start,exercise_end,utc_offset,exercise_created,exercise_last_updated,activity_name,log_type,pool_length,pool_length_unit,intervals_interval_type,intervals_interval_num,intervals_total_intervals,intervals_num_repeats,intervals_duration_millis,distance_units,tracker_total_calories,tracker_total_steps,tracker_total_distance_mm,tracker_total_altitude_mm,tracker_avg_heart_rate,tracker_peak_heart_rate,tracker_avg_pace_mm_per_second,tracker_avg_speed_mm_per_second,tracker_peak_speed_mm_per_second,tracker_auto_stride_run_mm,tracker_auto_stride_walk_mm,tracker_swim_lengths,tracker_pool_length,tracker_pool_length_unit,tracker_cardio_load,manually_logged_total_calories,manually_logged_total_steps,manually_logged_total_distance_mm,manually_logged_pool_length,manually_logged_pool_length_unit,exercise_event_id,event_timestamp,event_type,event_auto_cue_type,event_elapsed_time_millis,event_traveled_distance_mm,event_calories_burned,event_steps,event_average_heart_rate,event_elevation_gain_mm,event_swim_lengths,event_average_speed_mm_per_sec,event_interval_type
0,1630200506218544,2022-07-12 17:45:17+0000,2022-07-12 17:52:51+0000,+02:00,2022-07-12 18:10:43+0000,2022-07-12 18:10:49+0000,Walk,TRACKER,0,UNSPECIFIED,NaN,NaN,NaN,NaN,NaN,METRIC,63.0,785.0,469273.0,610.0,98.0,133.0,1038.0,1038.0,586.0,750.0,644.0,0.0,0.0,UNSPECIFIED,0.0,NaN,NaN,NaN,NaN,NaN,2.028137e+18,2022-07-12 17:52:49+0000,PAUSE,UNSPECIFIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNSPECIFIED
1,1630200506218544,2022-07-12 17:45:17+0000,2022-07-12 17:52:51+0000,+02:00,2022-07-12 18:10:43+0000,2022-07-12 18:10:49+0000,Walk,TRACKER,0,UNSPECIFIED,NaN,NaN,NaN,NaN,NaN,METRIC,63.0,785.0,469273.0,610.0,98.0,133.0,1038.0,1038.0,586.0,750.0,644.0,0.0,0.0,UNSPECIFIED,0.0,NaN,NaN,NaN,NaN,NaN,7.142397e+18,2022-07-12 17:52:51+0000,STOP,UNSPECIFIED,452026.0,469273.0,NaN,NaN,NaN,NaN,NaN,NaN,UNSPECIFIED
2,1630200506218544,2022-07-12 17:45:17+0000,2022-07-12 17:52:51+0000,+02:00,2022-07-12 18:10:43+0000,2022-07-12 18:10:49+0000,Walk,TRACKER,0,UNSPECIFIED,NaN,NaN,NaN,NaN,NaN,METRIC,63.0,785.0,469273.0,610.0,98.0,133.0,1038.0,1038.0,586.0,750.0,644.0,0.0,0.0,UNSPECIFIED,0.0,NaN,NaN,NaN,NaN,NaN,7.984065e+18,2022-07-12 17:45:17+0000,START,UNSPECIFIED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNSPECIFIED
3,3687920439982376,2022-03-04 08:24:42+0000,2022-03-04 08:28:27+0000,+01:00,2022-03-04 08:28:42+0000,2022-03-04 08:28:42+0000,Walk,TRACKER,0,UNSPECIFIED,NaN,NaN,NaN,NaN,NaN,METRIC,34.0,371.0,264414.0,13716.0,84.0,97.0,1180.0,1180.0,961.0,750.0,703.0,0.0,0.0,UNSPECIFIED,0.0,NaN,NaN,NaN,NaN,NaN,2.314016e+18,2022-03-04 08:28:27+0000,STOP,UNSPECIFIED,224025.0,264414.0,NaN,NaN,NaN,NaN,NaN,NaN,UNSPECIFIED


######
- Columnas Principales:
    - **exercise_id:**
	    Un identificador único para cada ejercicio registrado.
	    Es útil para diferenciar entre distintos ejercicios, especialmente si se registraron en el mismo día.
	- **exercise_start y exercise_end:**
	    Marcan la hora de inicio y fin del ejercicio, respectivamente.
	    Estas columnas se usan para calcular la duración total de cada sesión de ejercicio.
	- **utc_offset:**
	    Indica la diferencia horaria con respecto al Tiempo Universal Coordinado (UTC), que es útil para ajustar las horas al huso horario local.
	- **exercise_created y exercise_last_updated:**
	    Estas columnas muestran cuándo se creó y cuándo se actualizó por última vez el registro del ejercicio.
	    Sirven para rastrear cambios o ajustes en los datos del ejercicio.
	- **activity_name:**
	    El nombre de la actividad realizada, como “Walk” (Caminar), “Run” (Correr), etc.
	    Es crucial para identificar qué tipo de ejercicio fue realizado.
	- **log_type:**
	    Indica cómo se registró el ejercicio, por ejemplo, si fue manualmente (MANUALLY_LOGGED) o automáticamente por el dispositivo (TRACKER).
	- **pool_length y pool_length_unit:**
	    Estos datos son específicos para actividades de natación, donde se registra la longitud de la piscina y la unidad de medida.
	- **tracker_total_calories, tracker_total_steps, tracker_total_distance_mm:**
	    Proporcionan el total de calorías quemadas, pasos dados y distancia recorrida, respectivamente, durante el ejercicio.
	- **tracker_avg_heart_rate, tracker_peak_heart_rate:**
	    Muestran la frecuencia cardíaca promedio y máxima registrada durante el ejercicio.
	    Es valioso para analizar la intensidad del ejercicio.
	- **tracker_avg_pace_mm_per_second, tracker_avg_speed_mm_per_second, tracker_peak_speed_mm_per_second:**
	    Estos valores indican el ritmo promedio, velocidad promedio, y velocidad máxima durante el ejercicio.
	    Son útiles para evaluar el rendimiento en actividades de carrera o ciclismo.
	- **event_type y event_timestamp:**
	    event_type describe diferentes eventos ocurridos durante el ejercicio, como START, STOP, o PAUSE.
	    event_timestamp es la marca de tiempo de cada uno de estos eventos.

#### 01.01.16 journal_entries

- Contiene registros de entradas de diario que los usuarios han creado en la plataforma. Este tipo de archivo puede incluir datos personales y subjetivos relacionados con la salud, el estado de ánimo, actividades diarias, notas, etc.

- Este archivo podría incluir las siguientes columnas:
    - **entry_id:**
    	Un identificador único para cada entrada en el diario.
	- **date:**
		La fecha en la que se creó la entrada.
	- **time:**
		La hora en la que se registró la entrada.
	- **content:**
		El contenido de la entrada, donde el usuario pudo haber escrito libremente sobre su día, estado de ánimo, o cualquier otro aspecto que desee registrar.
	- **tags:**
		Palabras clave o categorías asignadas a la entrada para facilitar su organización y búsqueda.
	- **mood:**
		Si la plataforma incluye esta funcionalidad, podría haber un registro del estado de ánimo del usuario en el momento de hacer la entrada.
	- **location:**
		Información sobre el lugar donde se hizo la entrada, si está disponible.

In [64]:
path_journal_entries = 'Datos/Fitbit/Journal Log'

## Problemas con la lectura del csv, modificado mediante SublimeText seleccionando con esta expresión regular \{([^}]*)\} los metadatos entre llaves y cambiando las comas por puntos y comas
entradas_usuario = llamar_archivos(path_journal_entries, 'journal_entries fixed')
entradas_usuario.head(4)

,entry_id,log_time,log_type,value,meta,entry_category,zone_offset
0,a43539e0-e0b8-11ee-b777-714f14b810ef,2024-03-12T22:36:48,MOOD,1,{source=Stress; platform=mobile},MOOD,+01:00
1,7c20fac0-dc4e-11ee-b389-47915a5e2e74,2024-03-07T07:46:49,MOOD,1,{source=Stress; platform=mobile},MOOD,+01:00
2,80466e10-d004-11ee-8157-b3780a1de65d,2024-02-20T16:26:58,MOOD,3,{source=Stress; platform=mobile},MOOD,+01:00
3,c2a921d0-cfd2-11ee-b66b-61452618b639,2024-02-20T10:30:53,MOOD,3,{source=Stress; platform=mobile},MOOD,+01:00


#####
- **entry_id**
    Es un identificador único para cada entrada en el diario.
- **log_time:**
    Es el tiempo en el que se registró la entrada, en formato de fecha y hora.
- **log_type:**
    Tipo de entrada registrada, en este caso, todas parecen ser de tipo “MOOD”, lo que indica que se está registrando el estado de ánimo.
- **value:**
    Es un valor numérico asociado con la entrada. Dado que el log_type es “MOOD”, este valor podría representar un nivel o intensidad del estado de ánimo en una escala.
- **meta:**
    Contiene metadatos adicionales en forma de diccionario. En este caso, parece incluir la fuente del dato (por ejemplo, “Stress”) y la plataforma desde la que se registró (“mobile”).
- **entry_category:**
    Similar al log_type, categoriza la entrada; en este caso, es “MOOD”, alineado con log_type.
- **zone_offset:**
    Es el desplazamiento horario respecto a UTC, que indica la zona horaria en la que se hizo la entrada.

#### 01.01.17 Menstrual Health (Salud menstrual)

In [65]:
path_menstrual = 'Datos/Fitbit/Menstrual Health'

##### menstrual_health_birth_control

- Contiene información relacionada con los métodos de control de natalidad registrados en la aplicación Fitbit. Este tipo de datos es importante para el seguimiento de la salud menstrual y reproductiva. 

In [66]:
control_natal = llamar_archivos(path_menstrual, 'menstrual_health_birth_control')

control_natal.head(4)

,birth_control_type,event_date,has_started
0,Non-hormonal IUD,2018-05-09,True


######
- **birth_control_type (Tipo de Control de Natalidad):**
    Indica el tipo de método anticonceptivo que se ha registrado, en este caso, un DIU no hormonal (Non-hormonal IUD).
- **event_date (Fecha del Evento):**
    La fecha en la que se registró o comenzó el uso del método anticonceptivo, que en este caso es el 9 de mayo de 2018.
- **has_started (Ha Comenzado):**
    Un valor booleano (True o False) que indica si el método anticonceptivo ha comenzado a utilizarse. En este caso, está marcado como True, lo que significa que el método ha sido iniciado.

##### menstrual_health_settings

- Contiene configuraciones y preferencias relacionadas con el seguimiento de la salud menstrual establecida en la aplicación Fitbit. Esto podría incluir configuraciones como el ciclo menstrual, la duración de los periodos, y cualquier otra preferencia definida para personalizar la experiencia de seguimiento de la salud menstrual.

In [67]:
periodo = llamar_archivos(path_menstrual, 'menstrual_health_settings')

periodo.head(4)

,pregnancy_history,birth_control_history,avg_period_days,avg_cycle_days
0,False,False,6,28


###### 

- **pregnancy_history:**
	- Valor: False
	    Indica si sehas registrado un historial de embarazo en la aplicación. El valor False significa que no se ha registrado ningún historial de embarazo.
- **birth_control_history:**
	- Valor: False
	    Indica si se ha registrado un historial de uso de métodos anticonceptivos. El valor False significa que no se ha registrado ningún uso de anticonceptivos en la configuración.
- **avg_period_days:**
	- Valor: 6
	    Representa el número promedio de días que dura el periodo. En este caso, has establecido que el periodo dura, en promedio, 6 días.
- **avg_cycle_days:**
	- Valor: 28
	    Indica el número promedio de días que dura el ciclo menstrual completo, es decir, desde el inicio de un periodo hasta el inicio del siguiente. Aquí, el valor es de 28 días, lo que corresponde al ciclo menstrual típico.

##### menstrual_health_symptoms

- Contiene registros de los síntomas ingresados en la aplicación Fitbit relacionados con el ciclo menstrual. Estos datos son útiles para realizar un seguimiento de cómo varía la salud y bienestar a lo largo del ciclo.

In [68]:
sintomas_menstruales = llamar_archivos(path_menstrual, 'menstrual_health_symptoms')

sintomas_menstruales.head(4)

,timestamp,fluids,flow,conditions,sex,ovulation_test,cycle_altering_event,mood
0,2024-07-27,NaN,spotting,bloated,NaN,NaN,NaN,NaN
1,2024-07-26,NaN,spotting,NaN,NaN,NaN,NaN,NaN
2,2024-07-25,NaN,medium,NaN,NaN,NaN,NaN,NaN
3,2024-07-24,NaN,medium,NaN,NaN,NaN,NaN,fatigued


######

- **timestamp:**
    Fecha en la que se registrastó los síntomas. Esto permite ver cuándo se experimentastó cada uno de los síntomas o eventos registrados.
- **fluids:**
    Esta columna registraría la presencia de fluidos relacionados con el ciclo menstrual, como el moco cervical. Parece que no se han registrado datos para este campo en las entradas mostradas (todos son NaN).
- **flow:**
    Registra el flujo menstrual. Aquí ves valores como “spotting” (manchado leve) y “medium” (flujo medio). Esto permite seguir la intensidad del flujo en diferentes días.
- **conditions:**
    Esta columna registra condiciones relacionadas con el bienestar general, como “bloated” (hinchazón) o “fatigued” (fatiga).
- **sex:**
    Esta columna registraría la actividad sexual.
- **ovulation_test:**
    Registra los resultados de las pruebas de ovulación.
- **cycle_altering_event:**
    Aquí se registrarían eventos que podrían alterar el ciclo, como la toma de anticonceptivos o algún cambio hormonal significativo.
- **mood:**
    Esta columna refleja el estado de ánimo registrado, como “fatigued” (fatigada) en una de las entradas. Es útil para hacer un seguimiento de cómo el estado de ánimo puede variar en diferentes etapas del ciclo.


##### menstrual_health_cycles

- Contiene información detallada sobre los ciclos menstruales registrados en la aplicación de Fitbit. Este archivo incluye datos relacionados con las fechas y características de cada ciclo, lo que permite llevar un seguimiento detallado de la salud menstrual.

- Tipos de Datos en “menstrual_health_cycles”:

	- **start_date:**
        Fecha de inicio del ciclo menstrual.
	- **end_date:**
        Fecha de finalización del ciclo menstrual.
	- **duration:**
        Duración total del ciclo en días.
	- **period_length:**
        Duración del periodo dentro del ciclo, es decir, cuántos días duró la menstruación.
	- **is_period_predicted:**
        Indicador de si el periodo fue predicho por la aplicación o registrado manualmente.
	- **is_period_confirmed:**
        Indicador de si el periodo fue confirmado por el usuario.
	- **is_ovulation_day_predicted:**
        Indicador de si el día de ovulación fue predicho por la aplicación.
	- **is_ovulation_day_confirmed:**
        Indicador de si el día de ovulación fue confirmado por el usuario.

In [69]:
ciclos = llamar_archivos(path_menstrual, 'menstrual_health_cycles')
ciclos.head(4)

,id,cycle_start_date,cycle_end_date,ovulation_start_date,ovulation_end_date,ovulation_source,period_start_date,period_end_date,period_source,fertile_start_date,fertile_end_date,fertile_source
0,1476807,2018-04-26,2022-07-31,2022-07-18,2022-07-18,computed,2018-04-26,2018-04-30,manual,2022-07-13,2022-07-19,computed
1,216151738,2022-08-01,2022-08-28,2022-08-15,2022-08-15,computed,2022-08-01,2022-08-06,manual,2022-08-10,2022-08-16,computed
2,220304839,2022-08-29,2022-09-25,2022-09-12,2022-09-12,computed,2022-08-29,2022-09-03,manual,2022-09-07,2022-09-13,computed
3,225132356,2022-09-26,2022-10-23,2022-10-10,2022-10-10,computed,2022-09-26,2022-09-30,manual,2022-10-05,2022-10-11,computed


###### 

- ID del Ciclo **(id)**:
    Cada fila tiene un identificador único, lo que sugiere que cada fila representa un ciclo menstrual diferente para un individuo.
- Fechas del Ciclo:
	Inicio del Ciclo **(cycle_start_date)**:
        Marca la fecha en que comenzó el ciclo menstrual.
	Fin del Ciclo **(cycle_end_date)**:
        Indica cuándo terminó ese ciclo.
- Fechas de Ovulación:
	Inicio de la Ovulación **(ovulation_start_date)**:
        La fecha estimada o registrada en que comenzó la ovulación.
	Fin de la Ovulación **(ovulation_end_date)**:
        La fecha en que terminó la ovulación.
	Fuente de Ovulación **(ovulation_source)**:
        Indica si la información sobre la ovulación fue computada (estimada por un algoritmo) o registrada manualmente.
- Fechas del Periodo:
	Inicio del Periodo **(period_start_date)**:
        Fecha en la que comenzó el sangrado menstrual.
	Fin del Periodo **(period_end_date)**:
        Fecha en que finalizó el sangrado menstrual.
	Fuente del Periodo (period_source):
        Indica si la información del periodo fue registrada manualmente o estimada.
Fechas del Periodo Fértil:
    Inicio del Periodo Fértil **(fertile_start_date)**:
        Fecha estimada en la que comenzó el periodo fértil.
    Fin del Periodo Fértil **(fertile_end_date)** :
        Fecha en la que finalizó el periodo fértil.
    Fuente del Periodo Fértil **(fertile_source)**:
        Indica si esta información fue computada o registrada manualmente.

#### 01.01.18 Messages (No usado por privacidad)

#### 01.01.19 Mindfulness

In [70]:
path_mindfulness = 'Datos/Fitbit/Mindfulness'

##### Mindfulness Eda Data Sessions (Sin datos)

In [71]:
sesiones_eda_mindfulness = llamar_archivos(path_mindfulness, 'Mindfulness Eda Data Sessions')

sesiones_eda_mindfulness.head(4)

,session_id,timestamp,valid_data,activation,scl_avg


##### Mindfulness Goals

In [72]:
metas_mindfulnes = llamar_archivos(path_mindfulness, 'Mindfulness Goals')

metas_mindfulnes.head(4)

,date,days,default_goal
0,Thu Sep 24 00:00:00 UTC 2020,4,False


##### Mindfulness Sessions

- Contiene información detallada sobre sesiones de mindfulness registradas, incluyendo datos fisiológicos como la frecuencia cardíaca, así como detalles sobre la sesión en sí.

- **session_id:**
    Un identificador único para cada sesión de mindfulness. Este ID es importante para rastrear y diferenciar entre las distintas sesiones.
- **activity_name:**
    El nombre de la actividad de mindfulness realizada durante la sesión, como “Calmar la ansiedad” o “Calm Light”. Esto indica el tipo de ejercicio o práctica que se llevó a cabo.
- **average_heart_rate:**
    La frecuencia cardíaca promedio durante la sesión. Este dato es relevante para entender la respuesta fisiológica del cuerpo durante la práctica de mindfulness.
- **start_heart_rate:**
    La frecuencia cardíaca al inicio de la sesión. Puede utilizarse para comparar cómo varía la frecuencia cardíaca desde el comienzo hasta el final de la sesión.
- **end_heart_rate:**
    La frecuencia cardíaca al final de la sesión. Esto puede dar una idea de cómo la práctica de mindfulness afectó al individuo, particularmente en términos de relajación o reducción del estrés.
- **duration:**
    La duración total de la sesión en milisegundos. Este valor es clave para entender la longitud de cada sesión y para analizar cómo la duración puede influir en los resultados fisiológicos.
- **start_date_time:**
    La fecha y hora de inicio de la sesión. Esto puede ser útil para analizar patrones temporales, como la efectividad de las sesiones a diferentes horas del día.
- **end_date_time:**
    La fecha y hora de finalización de la sesión. Junto con la duración, permite confirmar la consistencia de los datos temporales.
- **session_type:**
    El tipo de sesión, que en este caso parece ser “mobile-discover”, posiblemente indicando que la sesión fue descubierta o iniciada a través de una aplicación móvil.
- **stress_metrics:**
    Aunque actualmente los valores son NaN, esta columna probablemente está destinada a contener métricas relacionadas con el estrés, que podrían ser calculadas a partir de otros datos o sensores.
- **pause_times:**
    También con valores NaN, esta columna podría registrar tiempos de pausa durante la sesión, si existieron.

In [73]:
sesiones_mindfulness = llamar_archivos(path_mindfulness, 'Mindfulness Sessions')

sesiones_mindfulness.head(4)

,session_id,activity_name,average_heart_rate,start_heart_rate,end_heart_rate,duration,start_date_time,end_date_time,session_type,stress_metrics,pause_times
0,b545f2e0-c41d-11ee-8080-808080808080,Calmar la ansiedad,47.0,55,46,20357,2024-02-05T12:57:16+01:00,2024-02-05T13:02:56+01:00,mobile-discover,NaN,NaN
1,7b6967f0-c1de-11ee-8080-808080808080,Calm Light,57.0,63,54,37105,2024-02-02T16:19:39+01:00,2024-02-02T16:29:59+01:00,mobile-discover,NaN,NaN
2,57056aa0-c119-11ee-8080-808080808080,Calm Light,57.0,65,53,37105,2024-02-01T16:48:27+01:00,2024-02-01T16:58:53+01:00,mobile-discover,NaN,NaN
3,08f99fc0-bf75-11ee-8080-808080808080,Calmar la ansiedad,56.0,59,52,20357,2024-01-30T14:39:47+01:00,2024-01-30T14:45:28+01:00,mobile-discover,NaN,NaN


#### 01.01.20 Oxigen Saturation (SpO2)

- La saturación de oxígeno (SpO2) es una métrica importante que mide el porcentaje de oxígeno en la sangre, y los datos de Fitbit sobre SpO2 pueden ser útiles para analizar la calidad del sueño, detectar posibles problemas respiratorios, entre otros aspectos de la salud.

In [74]:
path_oxigen = 'Datos/Fitbit/Oxygen Saturation (SpO2)'

##### Daily SpO2

- Contiene datos relacionados con la saturación de oxígeno en la sangre (SpO2) en un formato diario.

- **timestamp:**
    Esta columna contiene las fechas en formato de marca de tiempo (timestamp) en las que se registraron los datos diarios de SpO2.
- **average_value:**
	Esta columna muestra el valor promedio de saturación de oxígeno en la sangre para el día correspondiente. Este valor está en porcentaje (%), lo que indica qué porcentaje de oxígeno está presente en la sangre.
- **lower_bound:**
	El límite inferior de la saturación de oxígeno registrada durante el día. Este valor indica el nivel más bajo de SpO2 medido en ese día.
- **upper_bound:**
	El límite superior de la saturación de oxígeno registrada durante el día. Este valor representa el nivel más alto de SpO2 medido en ese día.

In [75]:
oxigeno_diario = llamar_archivos(path_oxigen, 'Daily SpO2')

oxigeno_diario.head(4)

,timestamp,average_value,lower_bound,upper_bound
0,2023-09-20T00:00:00Z,95.4,91.8,98.9
1,2023-09-21T00:00:00Z,96.3,93.5,98.8
2,2023-09-22T00:00:00Z,95.0,93.1,98.0
3,2023-09-23T00:00:00Z,95.8,92.3,99.8


##### Minute SpO2

- **timestamp:**
	Esta columna contiene las marcas de tiempo para cada medición, registradas a nivel de minuto. Cada fila representa un minuto específico en el que se midió la saturación de oxígeno.
- **value:**
	Este es el valor de la saturación de oxígeno (%) medido en el minuto correspondiente.

In [76]:
oxigeno_minutos = llamar_archivos(path_oxigen, 'Minute SpO2')

oxigeno_minutos.head(4)

,timestamp,value
0,2024-08-09T00:00:35Z,50.0
1,2024-08-09T00:01:35Z,50.0
2,2024-08-09T00:02:35Z,98.4
3,2024-08-09T00:03:35Z,50.0


#### 01.01.21 Paired Devices

- Contiene información sobre los dispositivos emparejados con las cuentas de Fitbit.

In [77]:
path_paired_devices = 'Datos/Fitbit/Paired Devices'

##### Devices

In [78]:
dispositivos = llamar_archivos(path_paired_devices, 'Devices')

dispositivos.head(4)

,wire_id,device_type,serial_number,enabled,fw_version
0,41B81230103D,Versa 4,NaN,False,APP61.20001.210.26 BSL61.0.0 PERIPH|BT_RADIO|1.0.0|0 PERIPH|BT_NFC|61.3.9|0 PERIPH|TOUCH|61.4.9|4 PERIPH|GPS|61.0.5|1 PERIPH|CODEC|61.5.0|0 PERIPH|FACTORY|3001.153.70|0 ASSETS|6|12192
1,f4c557e7d532,MobileTrack,32d5e757c5f4,False,APP50.0 BSL50.0


###### 
- **wire_id:**
    Este es un identificador único para cada dispositivo emparejado. Es similar a un “device ID” que se utiliza para identificar el dispositivo dentro del ecosistema Fitbit.
- **device_type:**
    Indica el tipo de dispositivo emparejado. Por ejemplo, “Versa 4” es un modelo de reloj inteligente de Fitbit, mientras que “MobileTrack” se refiere a la funcionalidad de seguimiento de actividad a través del teléfono móvil, sin necesidad de un dispositivo Fitbit.
- **serial_number:**
    El número de serie del dispositivo.
- **enabled:**
    Indica si el dispositivo está actualmente habilitado (activo) o no.
- **fw_version:**
    La versión del firmware del dispositivo.

##### iOS App Notification Settings

- **user_id:**
    Identificador único del usuario que está recibiendo las notificaciones. Esto permite asociar las notificaciones con un usuario específico en el sistema de Fitbit.
- **tracker_id:**
    Identificador del dispositivo Fitbit (tracker) al que están vinculadas las notificaciones. Esto permite relacionar las notificaciones con un dispositivo específico que esté emparejado con la cuenta del usuario.
- **mobile_app_name:**
	El nombre o identificador de la aplicación móvil que envía las notificaciones. Por ejemplo, “com.apple.AppStore” se refiere a la App Store de Apple, “com.apple.mobilemail” a la aplicación de correo de Apple, y “com.fitbit.FitbitMobile” a la propia aplicación de Fitbit.
- **is_app_enabled:**
	Indica si las notificaciones de la aplicación están habilitadas o no (True o False).
- **show_partial_message:**
	Indica si se permite mostrar mensajes parciales en las notificaciones (por ejemplo, una vista previa del contenido del mensaje).
- **is_default_message_app:**
	Indica si la aplicación es la predeterminada para manejar mensajes (por ejemplo, si es la aplicación principal para recibir notificaciones de mensajes de texto). Todos los valores son False, lo que sugiere que ninguna de estas aplicaciones es la predeterminada para mensajes.
- **created_on:**
	Fecha y hora en que se creó la configuración de notificaciones para esta aplicación. Este timestamp indica cuándo se configuró por primera vez esta opción.
- **modified_on:**
	Fecha y hora de la última modificación de la configuración de notificaciones para esta aplicación. Este timestamp indica cuándo se realizó el último cambio en la configuración.

In [79]:
notificaciones_app = llamar_archivos(path_paired_devices, 'iOS App Notification Settings')

notificaciones_app.head(4)

,user_id,tracker_id,mobile_app_name,is_app_enabled,show_partial_message,is_default_message_app,created_on,modified_on
0,62234717,2254831026,com.apple.AppStore,False,False,False,2023-03-21T10:56:05.000Z,2023-03-21T10:56:05.000Z
1,62234717,2254831026,com.apple.mobilemail,False,False,False,2023-11-20T09:25:39.000Z,2023-11-20T09:25:39.000Z
2,62234717,2254831026,com.fitbit.FitbitMobile,False,False,False,2023-03-20T20:03:17.000Z,2023-03-20T20:03:17.000Z
3,62234717,2254831026,com.apple.reminders,False,False,False,2023-05-21T09:37:27.000Z,2023-05-21T09:37:27.000Z


##### Scales

- Está relacionado con los datos de una balanza o dispositivo de medición corporal asociado a una cuenta de Fitbit.

- **scale_id:**
	Identificador único de la balanza o dispositivo de medición. Este ID sirve para identificar de manera única cada balanza asociada a la cuenta.
- **short_name:**
	Nombre corto o abreviado de la balanza o dispositivo. Esto podría ser una etiqueta amigable o una referencia rápida para identificar el dispositivo.
- **display_bf:**
	Indica si la balanza tiene la capacidad de mostrar la grasa corporal (Body Fat, BF).
- **display_bf_mass_unit:**
	Unidad de medida utilizada por la balanza para mostrar la masa de grasa corporal, si es que la balanza lo mide.
- **display_bmi:**
	Indica si la balanza tiene la capacidad de mostrar el índice de masa corporal (BMI).
- **user_icon_id:**
	Identificador del icono de usuario asociado con la balanza. Esto podría estar relacionado con la personalización de la balanza para diferentes usuarios (por ejemplo, diferentes miembros de la familia usando la misma balanza).

In [80]:
escalas = llamar_archivos(path_paired_devices, 'Scales')

escalas

,scale_id,short_name,display_bf,display_bf_mass_unit,display_bmi,user_icon_id


##### Tracker Optional Configuration

- **tracker_id:**
	Identificador único para cada rastreador Fitbit. Este ID es clave para distinguir entre diferentes dispositivos asociados a cada cuenta.
- **enabled_notification_types:**
	Lista de tipos de notificaciones que están habilitadas en el rastreador, como notificaciones de calendario, mensajes de texto, llamadas entrantes y notificaciones de aplicaciones.
- **on_right_hand:**
	Indica si el rastreador está configurado para usarse en la mano derecha (True o False).
- **clock_face:**
	Representa la cara del reloj que está configurada en el rastreador.
- **enable_inactivity_alerts:**
	Indica si las alertas de inactividad están habilitadas en el rastreador (True o False).
- **last_updated_ia_time:**
	Fecha y hora de la última actualización relacionada con alertas de inactividad. Este dato muestra cuándo se realizó la última actualización o cambio en estas alertas.
- **last_reboot_time:**
	Fecha y hora de la última vez que se reinició el rastreador.
- **payments_enabled:**
	Indica si el rastreador tiene habilitada la funcionalidad de pagos (True o False).
- **last_successful_wifi_connection_time:**
	Fecha y hora de la última conexión exitosa a una red Wi-Fi.
- **last_successful_wifi_connectionipv4addr:**
	Dirección IPv4 de la última conexión Wi-Fi exitosa.
- **last_successful_wifi_connectionipv6addr:**
	Dirección IPv6 de la última conexión Wi-Fi exitosa.
- **last_successful_wifi_connectionssid:**
	Nombre de la red Wi-Fi (SSID) a la que se conectó exitosamente por última vez el rastreador.
- **live_data_disabled:**
	Indica si la transmisión de datos en vivo está deshabilitada en el rastreador (True o False).

In [81]:
rastreador_config = llamar_archivos(path_paired_devices, 'Tracker Optional Configuration')

rastreador_config

,tracker_id,enabled_notification_types,on_right_hand,clock_face,enable_inactivity_alerts,last_updated_ia_time,last_reboot_time,payments_enabled,last_successful_wifi_connection_time,last_successful_wifi_connectionipv4addr,last_successful_wifi_connectionipv6addr,last_successful_wifi_connectionssid,live_data_disabled
0,2254831026,"[CALENDAR, TEXT_MESSAGE, INCOMING_CALL, APP_NOTIFICATIONS]",False,NaN,True,2024-07-26T15:06:47.000Z,NaN,False,NaN,NaN,NaN,NaN,NaN
1,2400523663,NaN,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN


##### Trackers

- **racker_id:**
	Identificador único para cada rastreador (tracker) Fitbit.
- **ate_added:**
	La fecha en que el rastreador fue añadido a la cuenta Fitbit.
- **ast_sync_date_time:**
	Fecha y hora de la última sincronización exitosa del dispositivo con la aplicación Fitbit.
- **att_level:**
	Nivel de batería del dispositivo, medido en porcentaje.
- **ardware_rev:**
	Revisión del hardware, que indica la versión de hardware del dispositivo.
- **s_display_distance:**
	Indica si la pantalla del dispositivo muestra la distancia recorrida (True o False).
- **s_display_calories:**
	Indica si la pantalla del dispositivo muestra las calorías quemadas (True o False).
- **s_display_clock:**
	Indica si la pantalla del dispositivo muestra la hora (True o False).
- **s_display_flower:**
	Probablemente relacionado con la visualización de alguna métrica o animación específica en el dispositivo. Es posible que esté asociado con características más antiguas o específicas de ciertos modelos Fitbit (True o False).
- **is_display_elevation:**
	Indica si la pantalla del dispositivo muestra la elevación (metros o pies ascendidos) (True o False).
- **is_display_chatter:**
	Podría estar relacionado con la visualización de notificaciones o interacciones en pantalla (True o False).
- **is_right_handed:**
	Indica si el dispositivo está configurado para usarse en la mano derecha (True) o izquierda (False).
- **tracker_name:**
	Nombre del dispositivo, como “Versa 4” o “MobileTrack”.
- **device_type:**
	Tipo de dispositivo, indicando si es un rastreador físico o una función como “MobileTrack” (seguimiento de actividad sin un dispositivo específico).
- **on_dominant_hand:**
	Indica si el dispositivo está configurado para usarse en la mano dominante (True o False).
- **is_display_active_minutes:**
	Indica si la pantalla del dispositivo muestra los minutos de actividad (True o False).
- **clock_face:**
	La cara del reloj seleccionada para mostrarse en el dispositivo.
- **enable_ancs:**
	Indica si está habilitada la función de notificaciones de aplicaciones (Apple Notification Center Service) (True o False).
- **is_bonded:**
	Indica si el dispositivo está emparejado de forma segura con tu cuenta o con un dispositivo móvil (True o False).
- **is_display_steps:**
	Indica si la pantalla del dispositivo muestra el conteo de pasos (True o False).
- **alarm_update_time:**
	Fecha y hora de la última actualización de la alarma del dispositivo.
- **is_display_heart_rate:**
	Indica si la pantalla del dispositivo muestra la frecuencia cardíaca (True o False).
- **heart_rate_tracking:**
	Método de seguimiento de la frecuencia cardíaca (por ejemplo, “AUTO”).
- **heart_rate_tracking_update_time:**
	Fecha y hora de la última actualización de la configuración de seguimiento de la frecuencia cardíaca.
- **tap_enabled:**
	Indica si la función de toque en la pantalla está habilitada (True o False).
- **tap_screen:**
	Tipo de respuesta al toque en la pantalla (por ejemplo, “TIME” podría indicar que un toque muestra la hora).
- **flick_enabled:**
	Indica si la función de “flick” (giro rápido de muñeca para activar la pantalla) está habilitada (True o False).
- **flick_screen:**
	Función o pantalla que se activa con el giro rápido de muñeca.

In [82]:
rastreador = llamar_archivos(path_paired_devices, 'Trackers')

rastreador

,tracker_id,date_added,last_sync_date_time,batt_level,hardware_rev,is_display_distance,is_display_calories,is_display_clock,is_display_flower,is_display_elevation,is_display_chatter,is_right_handed,tracker_name,device_type,on_dominant_hand,is_display_active_minutes,clock_face,enable_ancs,is_bonded,is_display_steps,alarm_update_time,is_display_heart_rate,heart_rate_tracking,heart_rate_tracking_update_time,tap_enabled,tap_screen,flick_enabled,flick_screen
0,2254831026,2023-03-20,2024-08-10T20:27:52.000Z,47,9,True,True,True,True,True,True,True,NaN,Versa 4,False,False,A,True,False,True,2023-03-20T19:10:23.000Z,True,AUTO,2023-03-20T19:10:23.000Z,True,TIME,False,NaN
1,2400523663,2023-10-02,2024-08-10T20:27:37.000Z,0,1,True,True,True,True,True,True,True,NaN,MobileTrack,False,False,A,True,False,True,2023-10-02T09:20:20.000Z,True,AUTO,2023-10-02T09:20:20.000Z,True,TIME,False,NaN


#### 01.01.22 Sleep (Profile)

- Contiene información detallada sobre los patrones de sueño a lo largo del tiempo. Cada fila representa un período de observación, y las columnas proporcionan métricas clave sobre la calidad y las características del sueño.

- **reation_date:**
	La fecha en la que se registró o creó el perfil de sueño. Esto indica cuándo se tomó la medición o análisis de los datos de sueño.
- **leep_type:**
	Tipo de perfil de sueño, en este caso, “Hedgehog” (Erizo). Este podría ser un nombre asignado por la aplicación Fitbit para categorizar el tipo de sueño basado en patrones observados.
- **eep_sleep:**
	Porcentaje de sueño profundo. El sueño profundo es crucial para la recuperación física y el funcionamiento óptimo del sistema inmunológico.
- **em_sleep:**
	Porcentaje de sueño REM (Rapid Eye Movement). El sueño REM es importante para la consolidación de la memoria y la salud mental.
- **leep_duration:**
	Duración total del sueño en horas.
- **leep_start_time:**
	Hora en la que comienza el sueño, probablemente medida desde el momento en que te quedas dormido hasta que te despiertas completamente.
- **chedule_variability:**
	Variabilidad en el horario de sueño, posiblemente midiendo cuán consistente es la hora de dormir y despertar de un día a otro.
- **estorative_sleep:**
	Porcentaje de sueño restaurador, que podría ser una métrica compuesta que combina varias fases del sueño para indicar cuán reparador fue el descanso.
- **ime_before_sound_sleep:**
	Tiempo (en minutos u horas) antes de comenzar a dormir profundamente o de manera estable.
- **sleep_stability:**
	Medida de la estabilidad del sueño, que podría estar relacionada con cuántas veces se despierta el usuario durante la noche o la regularidad de las fases de sueño.
- **nights_with_long_awakenings:**
	Número de noches en las que se experimentó despertares prolongados, que pueden interrumpir el descanso reparador.
- **days_with_naps:**
	Número de días en los que hubo siestas. Esto puede influir en la calidad y duración del sueño nocturno.

In [83]:
path_perfil_zzz = 'Datos/Fitbit/Sleep'

perfil_durmiendo = llamar_archivos(path_perfil_zzz, 'Sleep Profile')

perfil_durmiendo.head(4)

,creation_date,sleep_type,deep_sleep,rem_sleep,sleep_duration,sleep_start_time,schedule_variability,restorative_sleep,time_before_sound_sleep,sleep_stability,nights_with_long_awakenings,days_with_naps
0,2022-07-04,Hedgehog,56.33,15.54,4.92,24.67,42.79,67.42,14.56,3.87,10.71,32.14
1,2022-08-01,Hedgehog,46.43,13.43,5.29,24.79,43.78,67.28,21.35,4.68,6.45,35.48
2,2022-09-01,Hedgehog,52.54,15.33,4.96,24.80,44.22,63.94,17.98,4.09,10.34,34.48
3,2022-10-01,Hedgehog,49.52,16.77,4.61,25.10,44.28,64.67,12.69,4.21,17.86,28.57


#### 01.01.23 Sleep Score

- Contiene datos relacionados con la evaluación de la calidad del sueño en base a varias métricas registradas durante diferentes sesiones de sueño.

- **sleep_log_entry_id:**
	Identificador único para cada registro de sueño. Este ID es clave para identificar y diferenciar entre distintas sesiones de sueño.
- **timestamp:**
	Marca de tiempo que indica cuándo se registró el sueño o cuándo se evaluó la calidad del sueño. Especifica la fecha y hora exacta en formato UTC.
- **overall_score:**
	Puntaje general del sueño, que proporciona una evaluación general de la calidad del sueño en esa sesión específica.
- **composition_score:**
	Se refiere a un puntaje que evalúa la “composición” del sueño, es decir, cómo se distribuyen las diferentes fases del sueño (sueño ligero, profundo, y REM) durante una sesión de sueño. Este puntaje es una medida de la calidad del sueño basada en la proporción y distribución de estas fases.
- **revitalization_score:**
	Puntaje que indica cuán revitalizante o restaurador fue el sueño. Un puntaje más alto sugiere que el sueño fue más reparador.
- **duration_score:**
	Este puntaje, cuando está disponible, evalúa si la duración del sueño fue suficiente.
- **deep_sleep_in_minutes:**
	La cantidad de minutos pasados en sueño profundo durante la sesión. El sueño profundo es crucial para la recuperación física y la consolidación de la memoria.
- **resting_heart_rate:**
	Frecuencia cardíaca en reposo durante la sesión de sueño. Una frecuencia cardíaca más baja en reposo generalmente indica una buena condición cardiovascular y un sueño reparador.
- **restlessness:**
	Medida de la inquietud durante el sueño, expresada como un valor numérico. Un valor más alto podría indicar movimientos frecuentes o interrupciones durante el sueño, lo que podría afectar la calidad del descanso.

In [84]:
path_puntos_zzz = 'Datos/Fitbit/Sleep Score'

puntuacion_zzz = llamar_archivos(path_puntos_zzz, 'sleep_score')

puntuacion_zzz.head(4)

,sleep_log_entry_id,timestamp,overall_score,composition_score,revitalization_score,duration_score,deep_sleep_in_minutes,resting_heart_rate,restlessness
0,46484678887,2024-08-10T08:03:30Z,79,NaN,79,NaN,50.0,51,0.093544
1,46477883954,2024-08-09T06:59:30Z,72,NaN,72,NaN,56.0,50,0.130802
2,46465115105,2024-08-08T07:04:30Z,75,NaN,75,NaN,53.0,50,0.113976
3,46447388388,2024-08-06T07:03:30Z,65,NaN,65,NaN,68.0,46,0.138346


#### 01.01.24 Snore and Noise Detect

- Snore Details

    - Contiene datos relacionados con el nivel de ruido ambiental y la detección de eventos de ronquidos durante el sueño.

    - **timestamp:**
	    Marca de tiempo que indica cuándo se registró cada medición. Cada fila representa un intervalo específico en el que se midieron los niveles de ruido y se detectaron eventos de ronquidos.
    - **mean_dba:**
	    El nivel promedio de ruido en decibelios ajustados (dBA) durante el intervalo de tiempo. El dBA es una unidad que mide el nivel de presión sonora ajustada a la percepción del oído humano.
    - **max_dba:**
	    El nivel máximo de ruido en dBA registrado durante el intervalo de tiempo.
    - **min_dba:**
	    El nivel mínimo de ruido en dBA registrado durante el intervalo de tiempo.
    - **events_number:**
	    El número total de eventos de ruido detectados durante el intervalo. Esto podría incluir cualquier tipo de ruido detectado, no solo ronquidos.
    - **snoring_events_number:**
	    El número de eventos específicamente etiquetados como ronquidos durante el intervalo.
    - **snore_label:**
	    Un indicador que clasifica o etiqueta la presencia de ronquidos.
    - **sample_duration:**
	    Duración del intervalo de muestreo en segundos.

In [85]:
path_snore_noise = 'Datos/Fitbit/Snore and Noise Detect'

ronquidos_ruidos = llamar_archivos(path_snore_noise, 'Snore Details')

ronquidos_ruidos.head(4)

,timestamp,mean_dba,max_dba,min_dba,events_number,snoring_events_number,snore_label,sample_duration
0,2022-06-30T23:23:00,49,73,30,0,0,0,30
1,2022-06-30T23:23:30,30,31,30,0,0,0,30
2,2022-06-30T23:24:00,30,31,30,0,0,0,30
3,2022-06-30T23:24:30,30,31,30,0,0,0,30


#### 01.01.25 Social

- Función anonimizar_valor()

In [86]:
path_social = 'Datos/Fitbit/Social'

##### Fedd Cheers 

- Interacciones con otros usuarios en publicaciones dentro de la plataforma Fitbit. 

    - **post_id:**
        Identificador único para cada publicación en el feed social.
    - **comment_id:**
        Identificador de un comentario relacionado con la publicación.
    - **author_id:**
        Identificador único del usuario que realizó la acción (como dar un cheer).
    - **author_name:**
        Nombre del usuario que realizó la acción.

In [87]:
feed_cheers = llamar_archivos(path_social, 'Feed Cheers')

feed_cheers['author_id'] = feed_cheers['author_id'].apply(anonimizar_valor)

feed_cheers['author_name'] = feed_cheers['author_name'].apply(lambda x : f"Usuario{feed_cheers.index[feed_cheers['author_name'] == x][0] + 1}")

feed_cheers.head(4)

,post_id,comment_id,author_id,author_name
0,6b65c190-d609-11e7-9d74-f32448c3297b,NaN,76b7bb28f4af2cab9ce1cf04319f23f0606e3247ad3bcc5fdc06a4c14c4a477d,Usuario1
1,fe0ed4d0-f762-11e7-8910-a70b48445111,NaN,c1ead6bbfa6b1bc523ac9d29aa486d491fd9307ac1cf7de25ee34304a0d16199,Usuario2
2,c295d550-c247-11ee-b622-9933e63423c8,NaN,f8075ec11444b868c5951d79269e5c3c1a10960da7c2456533449769399294fc,Usuario3
3,30208bb0-c46e-11ee-ba25-eb76bd4884e1,NaN,c9be96ac50329ddb25074cb4613e20db0d4aca8ed7747c49545ec4b332526026,Usuario4


##### Feed Comments

- Contiene datos relacionados con comentarios realizados en una plataforma social, posiblemente dentro del ecosistema de Fitbit.

    - **id:**
        Este es el identificador único del comentario. Es un UUID (Identificador Único Universal), que asegura que cada comentario puede ser identificado de manera única dentro de la base datos.
    - **content:**
        Contenido del comentario. Esto es el texto que el usuario escribió como comentario. Puede incluir texto simple, emojis, y otros elementos de expresión como aplausos (👏).
    - **timestamp:**
        Marca de tiempo que indica cuándo fue realizado el comentario. Está en formato ISO 8601 con precisión hasta los milisegundos, lo que permite rastrear el momento exacto en que se publicó el comentario.
    - **post_id:**
        Este es el identificador del post al cual el comentario está asociado. Similar al id del comentario, es un UUID y permite enlazar el comentario a su post correspondiente.

In [88]:
comentarios = llamar_archivos(path_social, 'Feed Comments')

comentarios['id'] = comentarios['id'].apply(anonimizar_valor)
comentarios['post_id'] = comentarios['post_id'].apply(anonimizar_valor)

comentarios.head(2)

,id,content,timestamp,post_id
0,4e3fda5b62550d24c6629437e505f34610f1ffa63edc148cac2e5b3aec91e021,2,2024-04-03T21:40:51.891Z,d3eae812c78a283d720da306efac24320135bd767a5b92301671ddff728018f0
1,343f22ff7e5884797004b86bf0a7bf30d3b78a0bf86502a7e8d7de58a74e4f85,👏 bravo 👏,2019-02-12T10:48:43.093Z,f5429395273326ffdf2061f081110464c9f8d2776369053a906163cc85c4b028


##### Feed Posts

- Contiene información sobre las publicaciones realizadas por el usuario en la plataforma social.

In [89]:
mis_posts = llamar_archivos(path_social, 'Feed Posts')

mis_posts['id'] = mis_posts['id'].apply(anonimizar_valor)
mis_posts['group_id'] = mis_posts['group_id'].apply(anonimizar_valor)
mis_posts['image_filename'] = mis_posts['image_filename'].apply(anonimizar_valor)
mis_posts['embedded_url'] = mis_posts['embedded_url'].apply(anonimizar_valor)

mis_posts.head(2)

,id,text_content,timestamp,comment_count,cheer_count,type,group_id,image_filename,embedded_url
0,63815fb93d1de311e5d1a1ec260dede4efc31ff1a5c217a90f6a51f5e3e7fd18,NaN,2023-07-05T17:29:18.600Z,1,25,BADGE,NaN,03bfe693799c99cd78d10f7a46e58d6e8ef02ba59060a0151edaa527ac1a7da5,NaN
1,a97ed7a2b8262afa4cb5cc4811852922df68177e79da3501ee4bb550054340b8,👍 Ole p’a mi!!! 😂😂😂,2018-12-07T17:43:21.564Z,0,40,BADGE,NaN,157c7e362b9bd55bf599175121ad5bdd72d7cd6f7acb83bc4236513d261b95f5,NaN


######
- **id:**
	 Es el identificador único de cada post. Este valor es un UUID (Identificador Único Universal) que permite identificar de manera única cada publicación en la base de datos.
- **text_content:**
	 Contenido textual del post, es decir, lo que el usuario ha escrito. Esto puede incluir texto simple, emojis y otras expresiones.
- **timestamp:**
	 Marca de tiempo que indica cuándo fue creado el post. Está en formato ISO 8601 con precisión hasta los milisegundos, lo que permite rastrear el momento exacto de la publicación.
- **comment_count:**
	 El número de comentarios que ha recibido el post.
- **cheer_count:**
	 El número de “cheers” (reacciones positivas, similares a “likes”) que ha recibido el post.
- **type:**
	 Tipo de post. Puede indicar la naturaleza o categoría del post, como “BADGE” para posts relacionados con insignias o logros dentro de la plataforma.
- **group_id:**
	 Identificador del grupo al que pertenece el post, si corresponde.
- **image_filename:**
	 Nombre del archivo de imagen asociado con el post. Si el post incluye una imagen, este campo contendrá el nombre del archivo de esa imagen.
- **embedded_url:**
	 URL incrustada en el post, si corresponde. Puede ser un enlace a contenido externo relacionado con el post.

##### Groups

- Contiene información sobre diferentes grupos dentro de la plataforma social de Fitbit.

In [90]:
grupos = llamar_archivos(path_social, 'Groups')

grupos['id'] = grupos['id'].apply(anonimizar_valor)
grupos['avatar_url'] = grupos['avatar_url'].apply(anonimizar_valor)
grupos['cover_image_url'] = grupos['cover_image_url'].apply(anonimizar_valor)

grupos.head(2)

,id,name,description,avatar_url,cover_image_url,type,members
0,ed38423f7cadf697666621a9308dd50047d2c085b34a9b34d19ba5fd3e58ce42,Boxeo,Si te gusta el saco a saco este es tu grupo,8aa3ab4c9d2a85898a3b312c12ad079658e1418d6b23a5a1411b2fbf56faf7ce,8aa3ab4c9d2a85898a3b312c12ad079658e1418d6b23a5a1411b2fbf56faf7ce,Closed group,5
1,d2cae1537e67b55e5db5089c8874e85d2c34100583aa7e759e4acf6d35675b14,"FIT, ACTIVE & RESPECTFUL",Active people only. please help us keep trolls out 👍,4aaec7ba2994566aae307985a659a99fc7e856928173d667ee054d1da9b41ef3,4aaec7ba2994566aae307985a659a99fc7e856928173d667ee054d1da9b41ef3,Closed group,491


######
- **id:**
    Es el identificador único del grupo. Este valor es un UUID (Identificador Único Universal) que asegura que cada grupo puede ser identificado de manera única dentro de la plataforma.
- **name:**
    Nombre del grupo.
- **description:**
	Descripción del grupo. Proporciona información sobre el propósito del grupo, los temas que se discuten o los intereses compartidos por los miembros del grupo.
- **avatar_url:**
    URL del avatar del grupo. Esta columna contiene un enlace a la imagen que representa al grupo, utilizada como avatar o icono del grupo.
- **cover_image_url:**
    URL de la imagen de portada del grupo. Es el enlace a la imagen que se utiliza como portada del grupo, proporcionando una representación visual adicional del grupo.
- **type:**
    Tipo de grupo. Indica la naturaleza del grupo, como “Closed group”, lo que sugiere que es un grupo cerrado al que se necesita invitación o aprobación para unirse.
- **members:**
    Número de miembros del grupo. Esta columna muestra cuántas personas forman parte del grupo.

#### 01.01.26 Stress Journal (Sin datos)

#### 01.01.27 Stress Score

- Contiene información sobre los puntos relacionados con el estrés y otros aspectos de la salud diaria en la plataforma Fitbit.

In [91]:
path_stress = 'Datos/Fitbit/Stress Score'

stress_puntos = llamar_archivos(path_stress, 'Stress Score')

stress_puntos.head(4)

,DATE,UPDATED_AT,STRESS_SCORE,SLEEP_POINTS,MAX_SLEEP_POINTS,RESPONSIVENESS_POINTS,MAX_RESPONSIVENESS_POINTS,EXERTION_POINTS,MAX_EXERTION_POINTS,STATUS,CALCULATION_FAILED
0,2021-06-11T00:00:00,2021-06-11T16:33:56.8,66,0,0,0,0,0,0,READY_NOT_PREMIUM,False
1,2021-06-12T00:00:00,2021-06-12T19:30:41.302,0,0,0,0,0,0,0,NO_DATA,True
2,2021-06-13T00:00:00,2021-06-13T17:09:43.21,67,0,0,0,0,0,0,READY_NOT_PREMIUM,False
3,2021-06-14T00:00:00,2021-06-14T10:53:16.781,57,0,0,0,0,0,0,READY_NOT_PREMIUM,False


##### 
- **DATE:**
	-Fecha en la que se registran los puntos de estrés y otras métricas. Está en formato ISO 8601 (año-mes-día).
- **UPDATED_AT:**
	- Marca de tiempo que indica cuándo se actualizaron los datos para esa fecha en particular. Incluye la fecha y la hora.
- **STRESS_SCORE:**
	- Puntaje de estrés para el día. Este valor numérico indica el nivel de estrés.
- **SLEEP_POINTS:**
	-Puntos relacionados con la calidad del sueño. Estos puntos miden aspectos específicos del sueño, como su duración o efectividad.
- **MAX_SLEEP_POINTS:**
	- Valor máximo de puntos de sueño que se podrían haber obtenido. Esto representa el puntaje ideal o máximo posible para el sueño.
- **RESPONSIVENESS_POINTS:**
	- Puntos relacionados con la capacidad de respuesta o la recuperación. Mide cómo responde el cuerpo al estrés y otros factores a lo largo del día.
- **MAX_RESPONSIVENESS_POINTS:**
	- Valor máximo de puntos de capacidad de respuesta que se podrían haber obtenido. Representa el ideal o máximo posible.
- **EXERTION_POINTS:**
	- Puntos relacionados con el esfuerzo físico realizado durante el día. Mide la cantidad de actividad física y su impacto en el cuerpo.
- **MAX_EXERTION_POINTS:**
	- Valor máximo de puntos de esfuerzo físico que se podrían haber obtenido. Representa el ideal o máximo posible.
- **STATUS:**
	- Estado del puntaje de estrés, puede indicar si los datos están listos y si el usuario tiene una suscripción premium o no.
- **CALCULATION_FAILED:**
	- Indica si hubo un fallo en el cálculo de los puntos para ese día. Un valor de True significa que hubo un error en la obtención de los datos.

#### 01.01.27 Temperature

In [92]:
path_temperatura = 'Datos/Fitbit/Temperature'

##### Computed Temperature

- Contiene los siguientes datos relevantes sobre la temperatura registrada durante el sueño:

    - **type:**
        Parece ser un identificador que indica el tipo de dato, en este caso, “IDT” para todos los registros.
    - **sleep_start:**
        Fecha y hora de inicio del periodo de sueño.
    - **sleep_end:**
        Fecha y hora de finalización del periodo de sueño.
    - **temperature_samples:**
        Número de muestras de temperatura registradas durante el periodo de sueño.
    - **nightly_temperature:**
        Temperatura promedio registrada durante la noche.
    - **baseline_relative_sample_sum:**
        Suma de las desviaciones relativas de la temperatura respecto a un valor base.
    - **baseline_relative_sample_sum_of_squares:**
        Suma de los cuadrados de las desviaciones relativas.
    - **baseline_relative_nightly_standard_deviation:**
        Desviación estándar de la temperatura relativa durante la noche.
    - **baseline_relative_sample_standard_deviation:**
        Desviación estándar de las muestras relativas respecto al valor base.

In [93]:
medida_temp = llamar_archivos(path_temperatura, 'Computed Temperature')

medida_temp.head(4)

,type,sleep_start,sleep_end,temperature_samples,nightly_temperature,baseline_relative_sample_sum,baseline_relative_sample_sum_of_squares,baseline_relative_nightly_standard_deviation,baseline_relative_sample_standard_deviation
0,IDT,2023-04-01T02:40,2023-04-01T07:34:30,295,33.045424,116.885194,158.303690,0.300261,0.852341
1,IDT,2023-04-02T02:04:30,2023-04-02T10:36,510,32.532745,-40.943919,735.050225,0.287770,0.905345
2,IDT,2023-04-03T00:47,2023-04-03T07:02:30,371,32.964960,117.145786,374.954057,0.291174,0.918914
3,IDT,2023-04-03T23:36,2023-04-04T07:02:30,443,32.223702,-172.471091,364.008481,0.302210,0.913736


##### Device Temperature

- Es un registro de temperatura tomado por el dispositivo en intervalos de tiempo cortos.

    - **recorded_time:**
	    Marca de tiempo en la que se registró la temperatura.
    - **temperature:**
	    Valor de la temperatura registrado, que parece estar en una escala relativa (posiblemente en relación con un punto de referencia, como la temperatura corporal base).
    - **sensor_type:**
	    Tipo de sensor utilizado para registrar la temperatura.

In [94]:
dispositivo_temp = llamar_archivos(path_temperatura, 'Device Temperature')

dispositivo_temp.head(4)

/var/folders/t3/my7vf2lx5q31d5ffy_vh35qw0000gp/T/ipykernel_26588/380415358.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(lista, ignore_index = True)


,recorded_time,temperature,sensor_type
0,2024-06-27T00:00,-0.876833,UNKNOWN
1,2024-06-27T00:01,-0.776833,UNKNOWN
2,2024-06-27T00:02,-0.576833,UNKNOWN
3,2024-06-27T00:03,-0.476833,UNKNOWN


#### 01.01.28 Transactions (Sin datos)

#### 01.01.29 User Security Data

In [95]:
path_security = 'Datos/Fitbit/User Security Data'

##### User_Email_Audit_Entry

- **previous_email:**
	Destinada a contener las direcciones de correo electrónico anteriores del usuario antes de realizar algún cambio.
- **change_time:**
	Fecha y hora en la que se realizó el cambio de correo electrónico.
- **request_id:**
	Identificador único para la solicitud de cambio de correo electrónico.

In [96]:
email_usuario = llamar_archivos(path_security, 'User_Email_Audit_Entry')

email_usuario

,previous_email,change_time,request_id


##### User_Retired_Password

- Contiene información sobre los cambios de contraseña de un usuario.

    - **date_changed:**
        La fecha y hora en la que se realizó el cambio de contraseña.
	- **reason:**
        El motivo del cambio de contraseña.

In [97]:
usuario_password = llamar_archivos(path_security, 'User_Retired_Password')

usuario_password

,date_changed,reason
0,2019-12-12T06:35:28.000Z,120/FORGOT_PASSWORD
1,2022-01-17T15:44:48.000Z,120/FORGOT_PASSWORD
2,2024-01-18T08:13:18.000Z,160/GOOGLE_ACCOUNT_LINK


#### 01.01.29 Your Profile

- Contiene una variedad de información personal y de configuración relacionada con la cuenta y preferencias en la plataforma.

In [98]:
path_profile = 'Datos/Fitbit/Your Profile'

perfil_usuario = llamar_archivos(path_profile, 'Profile')

perfil_usuario['id'] = perfil_usuario['id'].apply(anonimizar_valor)
perfil_usuario['full_name'] = perfil_usuario['full_name'].apply(anonimizar_valor)
perfil_usuario['first_name'] = perfil_usuario['first_name'].apply(anonimizar_valor)
perfil_usuario['last_name'] = perfil_usuario['last_name'].apply(anonimizar_valor)
perfil_usuario['display_name'] = perfil_usuario['display_name'].apply(anonimizar_valor)
perfil_usuario['username'] = perfil_usuario['username'].apply(anonimizar_valor)
perfil_usuario['email_address'] = perfil_usuario['email_address'].apply(anonimizar_valor)


perfil_usuario

,id,full_name,first_name,last_name,display_name_setting,display_name,username,email_address,date_of_birth,child,country,state,city,timezone,locale,member_since,about_me,start_of_week,sleep_tracking,time_display_format,gender,height,weight,stride_length_walking,stride_length_running,weight_unit,distance_unit,height_unit,water_unit,glucose_unit,swim_unit
0,7a376ba7129bb22c861706a12adf2ced0fd756d1fdce84229bf70871e9223859,7cb4eb73cf7a244f3eb87416961c9c5656bd28bdd175fc8064bb691eef1b680d,be8da289072a17b5a6330ce109780537a5170feef164f7ec27a4f1ee8d89d127,d415d1f9d731150fd333b77170276c3a9575dbca6a2f1ffaa6f82629db1ca70d,username,08f31040bc2755e4b75ddd2bbff7c622be06b9eeb539b00a6d011cd6af99985a,08f31040bc2755e4b75ddd2bbff7c622be06b9eeb539b00a6d011cd6af99985a,9d2a24f0b1e9d1616c2949fb8e803e2c79f0fb292095c794b0a82562c2614493,1975-04-28,False,ES,NaN,NaN,Europe/Madrid,es_ES,2016-11-15,NaN,MONDAY,Sensitive,24hour,FEMALE,167.0,102.7,69.0,76.3,METRIC,METRIC,METRIC,METRIC,en_US,METRIC


### 02.01 EDA Apple

- Los datos de la carpeta de Apple que voy a utilizar son **bla_bla** y **bla_bla**. Los datos que no van a ser utilizados son **electrocardiograms** y **workout-routes**. Dado que mis predicciones en este trabajo van a basarse en otro tipo de información y la carga de los archivos en **electrocardiograms** daba problemas de Tokenización (número de columnas inconsistente) y la otra fuente de datos (Fitbit) no contiene electrocardiogramas, he optado por desechar estas carpetas y centrarme en los datos que si van a aportar valor.

In [99]:
path_apple = 'Datos/Apple'

#### export_cda

In [109]:
export_cda = llamar_archivos(path_apple, 'export_cda')

export_cda.head(4)

,code,root,extension,codeSystem,codeSystemName,displayName,title,value,patientRole,section
0,US,None,None,None,None,None,None,None,None,None
1,None,2.16.840.1.113883.1.3,POCD_HD000040,None,None,None,None,None,None,None
2,None,2.16.840.1.113883.10.20.22.1.2,None,None,None,None,None,None,None,None
3,None,1.1.1.1.1.1.1.1.1,Health Export CDA,None,None,None,None,None,None,None


#### exportacion

- Contiene datos exportados del dispositivo Apple Watch.

    - **value:**
	Contiene el valor medido o registrado, que puede ser una medida específica como la glucosa en sangre, el consumo de agua, entre otros.
    - **HKCharacteristicTypeIdentifierDateOfBirth:**
        Fecha de nacimiento del usuario.
    - **HKCharacteristicTypeIdentifierBiologicalSex:**
        Sexo biológico del usuario (por ejemplo, femenino o masculino).
    - **HKCharacteristicTypeIdentifierBloodType:**
        Tipo de sangre del usuario (NotSet).
    - **HKCharacteristicTypeIdentifierFitzpatrickSkinType:**
        Clasificación del tipo de piel del usuario según la escala Fitzpatrick (no definido).
    - **type:**
        Tipo de métrica registrada, como glucosa en sangre o agua ingerida.
    - **sourceName:**
        Nombre de la aplicación o dispositivo que registró los datos (por ejemplo, “Salud”, “YAZIO”).
    - **sourceVersion:**
        Versión de la aplicación o dispositivo que registró los datos.
    - **unit:**
        Unidad de medida utilizada para el valor registrado (por ejemplo, mg/dL para glucosa, mL para agua).
    - **creationDate,**
        startDate, endDate: Fechas y horas relacionadas con la creación y el periodo de la medición.
    - **MetadataEntry,**
        device, workoutActivityType, duration: Columnas que podrían contener datos adicionales sobre los dispositivos utilizados, eventos de entrenamiento, y otras estadísticas, aunque en muchos casos están vacías o no aplican.


In [124]:
exportacion_apple = llamar_archivos(path_apple, 'exportacion.xml')

exportacion_apple.head(4)

,value,HKCharacteristicTypeIdentifierDateOfBirth,HKCharacteristicTypeIdentifierBiologicalSex,HKCharacteristicTypeIdentifierBloodType,HKCharacteristicTypeIdentifierFitzpatrickSkinType,HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,MetadataEntry,device,workoutActivityType,duration,durationUnit,WorkoutEvent,WorkoutStatistics,WorkoutRoute,dateComponents,activeEnergyBurned,activeEnergyBurnedGoal,activeEnergyBurnedUnit,appleMoveTime,appleMoveTimeGoal,appleExerciseTime,appleExerciseTimeGoal,appleStandHours,appleStandHoursGoal,HeartRateVariabilityMetadataList
0,2024-08-08 18:23:50 +0200,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
1,None,1975-04-28,HKBiologicalSexFemale,HKBloodTypeNotSet,HKFitzpatrickSkinTypeNotSet,No hay,None,None,None,None,None,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
2,121.105,None,None,None,None,None,HKQuantityTypeIdentifierBloodGlucose,YAZIO,483,mg/dL,2019-08-22 22:13:11 +0200,2019-08-22 22:12:57 +0200,2019-08-22 22:12:57 +0200,NaN,None,None,NaN,None,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
3,97,None,None,None,None,None,HKQuantityTypeIdentifierBloodGlucose,Salud,17.1.2,mg/dL,2023-12-19 09:22:39 +0200,2023-12-19 09:22:00 +0200,2023-12-19 09:22:00 +0200,NaN,None,None,NaN,None,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None


In [126]:
datos_apple = exportacion_apple.copy()
datos_apple.head(4)

,value,HKCharacteristicTypeIdentifierDateOfBirth,HKCharacteristicTypeIdentifierBiologicalSex,HKCharacteristicTypeIdentifierBloodType,HKCharacteristicTypeIdentifierFitzpatrickSkinType,HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,MetadataEntry,device,workoutActivityType,duration,durationUnit,WorkoutEvent,WorkoutStatistics,WorkoutRoute,dateComponents,activeEnergyBurned,activeEnergyBurnedGoal,activeEnergyBurnedUnit,appleMoveTime,appleMoveTimeGoal,appleExerciseTime,appleExerciseTimeGoal,appleStandHours,appleStandHoursGoal,HeartRateVariabilityMetadataList
0,2024-08-08 18:23:50 +0200,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
1,None,1975-04-28,HKBiologicalSexFemale,HKBloodTypeNotSet,HKFitzpatrickSkinTypeNotSet,No hay,None,None,None,None,None,None,None,NaN,None,None,NaN,None,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
2,121.105,None,None,None,None,None,HKQuantityTypeIdentifierBloodGlucose,YAZIO,483,mg/dL,2019-08-22 22:13:11 +0200,2019-08-22 22:12:57 +0200,2019-08-22 22:12:57 +0200,NaN,None,None,NaN,None,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None
3,97,None,None,None,None,None,HKQuantityTypeIdentifierBloodGlucose,Salud,17.1.2,mg/dL,2023-12-19 09:22:39 +0200,2023-12-19 09:22:00 +0200,2023-12-19 09:22:00 +0200,NaN,None,None,NaN,None,None,NaN,None,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None


In [127]:
datos_apple.columns

Index(['value', 'HKCharacteristicTypeIdentifierDateOfBirth',
       'HKCharacteristicTypeIdentifierBiologicalSex',
       'HKCharacteristicTypeIdentifierBloodType',
       'HKCharacteristicTypeIdentifierFitzpatrickSkinType',
       'HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse', 'type',
       'sourceName', 'sourceVersion', 'unit', 'creationDate', 'startDate',
       'endDate', 'MetadataEntry', 'device', 'workoutActivityType', 'duration',
       'durationUnit', 'WorkoutEvent', 'WorkoutStatistics', 'WorkoutRoute',
       'dateComponents', 'activeEnergyBurned', 'activeEnergyBurnedGoal',
       'activeEnergyBurnedUnit', 'appleMoveTime', 'appleMoveTimeGoal',
       'appleExerciseTime', 'appleExerciseTimeGoal', 'appleStandHours',
       'appleStandHoursGoal', 'HeartRateVariabilityMetadataList'],
      dtype='object')

In [130]:
total_nulos = datos_apple.isnull().sum()
print(f"Total de valores nulos en el DataFrame: {total_nulos}")


Total de valores nulos en el DataFrame: value                                                            734
HKCharacteristicTypeIdentifierDateOfBirth                    1028956
HKCharacteristicTypeIdentifierBiologicalSex                  1028956
HKCharacteristicTypeIdentifierBloodType                      1028956
HKCharacteristicTypeIdentifierFitzpatrickSkinType            1028956
HKCharacteristicTypeIdentifierCardioFitnessMedicationsUse    1028956
type                                                             735
sourceName                                                       305
sourceVersion                                                  11921
unit                                                           11390
creationDate                                                     305
startDate                                                        305
endDate                                                          305
MetadataEntry                                                10

In [137]:
for columna in datos_apple.columns:
    print(f'"{columna}".info()')
    print(datos_apple[columna].info())
    print('\n')

"value".info()
<class 'pandas.core.series.Series'>
RangeIndex: 1028957 entries, 0 to 1028956
Series name: value
Non-Null Count    Dtype 
--------------    ----- 
1028223 non-null  object
dtypes: object(1)
memory usage: 7.9+ MB
None


"HKCharacteristicTypeIdentifierDateOfBirth".info()
<class 'pandas.core.series.Series'>
RangeIndex: 1028957 entries, 0 to 1028956
Series name: HKCharacteristicTypeIdentifierDateOfBirth
Non-Null Count  Dtype 
--------------  ----- 
1 non-null      object
dtypes: object(1)
memory usage: 7.9+ MB
None


"HKCharacteristicTypeIdentifierBiologicalSex".info()
<class 'pandas.core.series.Series'>
RangeIndex: 1028957 entries, 0 to 1028956
Series name: HKCharacteristicTypeIdentifierBiologicalSex
Non-Null Count  Dtype 
--------------  ----- 
1 non-null      object
dtypes: object(1)
memory usage: 7.9+ MB
None


"HKCharacteristicTypeIdentifierBloodType".info()
<class 'pandas.core.series.Series'>
RangeIndex: 1028957 entries, 0 to 1028956
Series name: HKCharacteristicTypeId

### 03.01 EDA datos manuales

In [102]:
pesos_manual = pd.read_csv('Datos/Manual/pesos_manuales.csv')

# La columna desnuda está en 1 para si y en 0 para vestida
pesos_manual.tail(4)

,fecha,peso,desnuda
79,2/09/2024,101.0,1
80,2/09/2024,102.2,0
81,3/09/2024,101.3,1
82,3/09/2024,102.3,0


In [103]:
col_peso_manual = {
    'fecha': 'Fecha',
    'peso': 'Peso',
    'desnuda': 'Domicilio'
}

In [104]:
pesos_manual.rename(columns = col_peso_manual, inplace = True)
pesos_manual.sort_values(by = 'Fecha', ascending = True)
pesos_manual.head(4)

,Fecha,Peso,Domicilio
0,8/12/2023,119.9,0
1,5/12/2023,121.4,0
2,9/12/2023,120.2,0
3,16/12/2023,119.9,0


In [105]:
pesos_manual.reset_index(drop = True, inplace = True)
pesos_manual.head(4)

,Fecha,Peso,Domicilio
0,8/12/2023,119.9,0
1,5/12/2023,121.4,0
2,9/12/2023,120.2,0
3,16/12/2023,119.9,0


In [106]:
# Agrupar por 'Fecha' y calcular la media de 'Peso'
pesos_manual_media_diaria = pesos_manual.groupby('Fecha', as_index = False).mean()

pesos_manual_media_diaria.head(4)


,Fecha,Peso,Domicilio
0,04/02/2024,113.6,1.0
1,06/4/2025,106.7,0.0
2,1/09/2024,101.5,0.5
3,1/4/2024,106.9,0.0


In [107]:
pesos_manual_media_diaria.drop('Domicilio', axis = 1, inplace = True)
pesos_manual_media_diaria.head(4)

,Fecha,Peso
0,04/02/2024,113.6
1,06/4/2025,106.7
2,1/09/2024,101.5
3,1/4/2024,106.9


In [108]:
pesos_manual_ok = pesos_manual_media_diaria.copy()
pesos_manual_ok.sample(4)

,Fecha,Peso
54,9/12/2023,120.2
13,17/07/2024,101.9
38,26/08/2024,100.4
52,8/12/2023,119.9


### 01.04 Concatenación Dataframes para el dataframe final con todos los datos a utilizar para nuestra predicción
    - pesos_manual_ok
    -